# Original best performing feature subset with ridge regression 

In [38]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from copy import copy
import scipy
import pickle

# %% Configuration
# Configuration parameters
droprows = 7050
std_reg_const = 0.1
normalization_std_reg = 0.0001
ridge_alpha = 1333

selected_cols = [
    # Opening/Closing difference features
    'xdiff_from_opening',
    
    # X-price features
    'xlog',
    'xlog_dayly_ewma_10min',
    'xprice_dayly_ewma_4hour',
    'xprice_lag_4hours',
    'xprice_time_mean_1hour_lag_4hours',
    'xprice_time_mean_10min_dayly_ewma_10min',
    
    # Y-price features
    'ylog_dayly_ewma_1hour',
    'yprice_dayly_ewma_10min',
    'yprice_ewma_difpair_10min_4hour',
    'yprice_expanding_mean_diff',
    'yprice_time_mean_1hour',
    'yprice_time_mean_1hour_lag_1workweek',
    'yprice_time_mean_10min',
    'yprice_time_mean_10min_dayly_ewma_4hour',
    'yprice_time_mean_10min_lag_10min',
    'yprice_time_mean_10min_rsi_1hour',
    'yprice_time_mean_2hours',
    'yprice_time_zscore_1hour',
    'yprice_time_zscore_2hours',
    
    # XY combined features
    'xy_garmonic_time_std_4hours',
    'xy_geom_time_mean_2hours_dayly_ewma_20min',
    'xy_square_time_zscore_10min',
    
    # YX spread features
    'yx_spread_ewma_prodpair_1hour_10min',
    'yx_spread_time_mean_10min_lag_1hour',
    'yx_spread_time_zscore_4hours',
]


# %% Helper Functions
def print_importances(model, selected_cols):
    """Print feature importances sorted by absolute weight"""
    weigts_sum = sum(map(abs, model.coef_))
    for name, weight in sorted(zip(selected_cols, model.coef_), key=lambda x: -abs(x[1])):
        percent_weight = abs(weight) / weigts_sum
        print('{:40} {:.2%} {:15.2}'.format(name, percent_weight, weight))

def rsquared(x, y):
    """Return R^2 where x and y are array-like."""
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

def rsiFunc(prices, n=14):
    """Calculate RSI (Relative Strength Index)"""
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1]
        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n
        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)
    return rsi

# %% Time Series Validation Functions
def time_split(data, valid_ratio, test_ratio):
    """Split time series data into train, validation, and test sets"""
    n_valid = max(1, int(data.shape[0] * valid_ratio))
    n_test = max(1, int(data.shape[0] * test_ratio))
    n_train = data.shape[0] - n_valid - n_test
    
    train = data.iloc[:n_train].reset_index(drop=True).copy()
    valid = data.iloc[n_train:-n_test].reset_index(drop=True).copy()
    test = data.iloc[-n_test:].reset_index(drop=True).copy()
    merged_test = pd.concat([valid, test], ignore_index=True)
    return train, valid, test

def validate_model_by_pentate(model, source_data, base_cols, droprows=0):
    """Validate model using 5-fold time series cross-validation"""
    df = source_data.copy()
    selected_cols = base_cols.copy()
    helper_cols = list(set(selected_cols + ['periods_before_closing', 'returns']))
    metrics_dict = {}
    
    for step in range(5, 10):
        n_train = int(df.shape[0] * step // 10)
        n_test = int(df.shape[0] * (step + 1) // 10)
        train = df.iloc[:n_train].reset_index(drop=True).copy()
        test = df.iloc[n_train:n_test].reset_index(drop=True).copy()
        train.drop(np.arange(droprows), inplace=True)
        train.dropna(inplace=True)

        model.fit(train[selected_cols], train.returns)
        predicted = model.predict(test[selected_cols])
        predicted[test.periods_before_closing == 0] = 0

        current_mse = mean_squared_error(test.returns, predicted)
        current_r2 = rsquared(test.returns, predicted) * 100
        metrics_dict['train_{}_percent'.format(step * 10)] = {
            'mse': current_mse,
            'r2': current_r2
        }
    
    report = pd.DataFrame(metrics_dict)
    report['min_stats'] = report.iloc[:,:5].min(1).astype(np.float32)
    report['max_stats'] = report.iloc[:,:5].max(1).astype(np.float32)
    report['avg'] = report.mean(1).astype(np.float32)
    return report

# %% TRULY FIXED Feature Engineering Function
def create_all_features(data, is_train=True):
    """
    TRULY FIXED: Feature engineering with NO data leakage.
    All features use strictly backward-looking rolling windows.
    Current observation at time t IS included (since we predict t+60).
    No resample+ffill pattern that causes intra-bin leakage.
    """
    print(f"Creating all features (NO LEAKAGE - {'TRAIN' if is_train else 'TEST'})...")
    
    # Pre-calculate commonly used values
    days = data.day.unique()
    
    # Opening/Closing differences (using previous day's close)
    print("Creating opening/closing difference features...")
    close_price_per_day_y = data.groupby('day').timestamp.max().shift(1).map(
        data[['timestamp', 'yprice']].set_index('timestamp').yprice)
    data['ydiff_from_closing'] = (data.yprice - data.day.map(close_price_per_day_y)).fillna(0)
    
    close_price_per_day_x = data.groupby('day').timestamp.max().shift(1).map(
        data[['timestamp', 'xprice']].set_index('timestamp').xprice)
    data['xdiff_from_closing'] = (data.xprice - data.day.map(close_price_per_day_x)).fillna(0)
    
    open_price_per_day_x = data.groupby('day').timestamp.min().map(
        data[['timestamp', 'xprice']].set_index('timestamp').xprice)
    data['xdiff_from_opening'] = data.xprice - data.day.map(open_price_per_day_x)
    
    # Log features
    print("Creating log features...")
    data['xlog'] = data.xprice.apply(np.log1p)
    data['ylog'] = data.yprice.apply(np.log1p)
    
    # Expanding mean that only uses past data (including current)
    print("Creating expanding mean difference...")
    data['yprice_expanding_mean_diff'] = data['yprice'] - data['yprice'].expanding(min_periods=1).mean()
    
    # FIXED: Time-based rolling features with backward-looking windows INCLUDING current
    print("Creating time-based rolling features (TRULY FIXED)...")
    time_windows = {
        6: '1min', 60: '10min', 360: '1hour', 720: '2hours', 1410: '4hours', 2820: '1workweek'
    }
    
    # Create rolling means with backward-looking windows INCLUDING current observation
    for window, name in time_windows.items():
        if window in [6, 60, 360, 1410]:  # xprice windows
            colname = f'xprice_time_mean_{name}'
            # Rolling includes current observation (no shift)
            data[colname] = data['xprice'] - data['xprice'].rolling(
                window=window, min_periods=1).mean()
        
        if window in [60, 360, 720]:  # yprice windows
            colname = f'yprice_time_mean_{name}'
            # Rolling includes current observation (no shift)
            data[colname] = data['yprice'] - data['yprice'].rolling(
                window=window, min_periods=1).mean()
    
    # Create rolling std for yprice with backward-looking windows INCLUDING current
    for window in [360, 720]:
        name = time_windows[window]
        colname = f'yprice_time_std_{name}'
        # Rolling includes current observation (no shift)
        data[colname] = data['yprice'].rolling(
            window=window, min_periods=1).std().fillna(0) + std_reg_const
    
    # Z-scores for yprice
    data['yprice_time_zscore_1hour'] = data.yprice_time_mean_1hour / data.yprice_time_std_1hour
    data['yprice_time_zscore_2hours'] = data.yprice_time_mean_2hours / data.yprice_time_std_2hours
    
    # Intraday EWMA features (backward-looking INCLUDING current)
    print("Creating intraday EWMA features...")
    ewma_configs = [
        ('xprice', [24], ['4hour']),
        ('xlog', [60], ['10min']),
        ('ylog', [360], ['1hour']),
        ('yprice', [24, 60], ['4hour', '10min'])
    ]
    
    for col, windows, names in ewma_configs:
        for day in days:
            df_mask = (data.day == day)
            day_data = data.loc[df_mask].copy()
            
            for window, name in zip(windows, names):
                colname = f'{col}_dayly_ewma_{name}'
                # EWMA includes current observation (no shift)
                ewm = day_data[col].ewm(halflife=window, adjust=False).mean()
                
                if col in ['xprice', 'yprice']:
                    data.loc[df_mask, colname] = day_data[col] - ewm
                elif col in ['xlog', 'ylog']:
                    data.loc[df_mask, colname] = day_data[col.replace('log', 'price')] - ewm
    
    # EWMA difference pair
    data['yprice_ewma_difpair_10min_4hour'] = data.yprice_dayly_ewma_10min - data.yprice_dayly_ewma_4hour
    
    # Lag features (inherently backward-looking, excludes current)
    print("Creating lag features...")
    lag_configs = [
        ('xprice', 1410, '4hours'),
        ('yprice', 1410, '4hours'),
        ('xprice_time_mean_1hour', 1410, '4hours'),
        ('yprice_time_mean_1hour', 1410, '4hours'),
        ('yprice_time_mean_1hour', 2820, '1workweek'),
        ('yprice_time_mean_10min', 60, '10min')
    ]
    
    for col, lag, name in lag_configs:
        data[f'{col}_lag_{name}'] = data[col].shift(lag)
    
    # RSI features (calculated on backward-looking data including current)
    print("Creating RSI features...")
    # RSI needs some history, so we calculate it on the rolling means
    if 'xprice_time_mean_1min' in data.columns:
        data['xprice_time_mean_1min_rsi_1min'] = rsiFunc(data['xprice_time_mean_1min'].fillna(0).values, 6)
    data['yprice_time_mean_10min_rsi_1hour'] = rsiFunc(data['yprice_time_mean_10min'].fillna(0).values, 360)
    
    # Additional EWMA on time means
    print("Creating EWMA on time means...")
    ewma_on_means = [
        ('xprice_time_mean_10min', 60, '10min'),
        ('yprice_time_mean_10min', 24, '4hour')
    ]
    
    for col, window, name in ewma_on_means:
        for day in days:
            df_mask = (data.day == day)
            day_data = data.loc[df_mask].copy()
            colname = f'{col}_dayly_ewma_{name}'
            # EWMA includes current observation (no shift)
            ewm = day_data[col].ewm(halflife=window, adjust=False).mean()
            data.loc[df_mask, colname] = ewm
    
    # XY Combined features with backward-looking windows INCLUDING current
    print("Creating XY combined features...")
    
    # XY Harmonic std - use rolling window INCLUDING current
    data['xy_garmonic_time_std_4hours'] = (
        data['xy_garmonic'].rolling(window=1410, min_periods=1).std().fillna(0) + std_reg_const
    )
    
    # Harmonic EWMA
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'xy_garmonic_dayly_ewma_1hour'] = (
            day_data['xy_garmonic'].ewm(halflife=360, adjust=False).mean()
        )
        data.loc[df_mask, 'xy_garmonic_dayly_ewma_2hours'] = (
            day_data['xy_garmonic'].ewm(halflife=720, adjust=False).mean()
        )
    
    data['xy_garmonic_dayly_ewma_1hour'] = data['xy_garmonic'] - data['xy_garmonic_dayly_ewma_1hour']
    data['xy_garmonic_dayly_ewma_2hours'] = data['xy_garmonic'] - data['xy_garmonic_dayly_ewma_2hours']
    data['xy_garmonic_ewma_prodpair_2hours_1hour'] = (
        data.xy_garmonic_dayly_ewma_2hours * data.xy_garmonic_dayly_ewma_1hour)
    
    # XY Geometric features with rolling windows INCLUDING current
    for window, name in [(6, '1min'), (60, '10min'), (360, '1hour'), (720, '2hours')]:
        colname = f'xy_geom_time_mean_{name}'
        # Rolling includes current observation (no shift)
        data[colname] = data['xy_geom'] - data['xy_geom'].rolling(
            window=window, min_periods=1).mean()
    
    # Geometric lags and EWMA
    data['xy_geom_time_mean_1hour_lag_20min'] = data['xy_geom_time_mean_1hour'].shift(120)
    
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'xy_geom_time_mean_10min_dayly_ewma_1min'] = (
            day_data['xy_geom_time_mean_10min'].ewm(halflife=6, adjust=False).mean()
        )
        data.loc[df_mask, 'xy_geom_time_mean_2hours_dayly_ewma_20min'] = (
            day_data['xy_geom_time_mean_2hours'].ewm(halflife=120, adjust=False).mean()
        )
    
    # XY Relation std with rolling windows INCLUDING current
    for window, name in [(360, '1hour'), (720, '2hours')]:
        colname = f'xy_relation_time_std_{name}'
        data[colname] = (
            data['xy_relation'].rolling(window=window, min_periods=1).std().fillna(0) + std_reg_const
        )
    
    # XY Square zscore with rolling windows INCLUDING current
    data['xy_square_time_mean_10min'] = (
        data['xy_square'] - data['xy_square'].rolling(window=60, min_periods=1).mean()
    )
    data['xy_square_time_std_10min'] = (
        data['xy_square'].rolling(window=60, min_periods=1).std().fillna(0) + std_reg_const
    )
    data['xy_square_time_zscore_10min'] = data['xy_square_time_mean_10min'] / data['xy_square_time_std_10min']
    
    # YX Spread features with rolling windows INCLUDING current
    print("Creating YX spread features...")
    for window, name in [(60, '10min'), (720, '2hours'), (1410, '4hours')]:
        colname = f'yx_spread_time_mean_{name}'
        data[colname] = (
            data['yx_spread'] - data['yx_spread'].rolling(window=window, min_periods=1).mean()
        )
    
    # Spread std and zscore with rolling windows INCLUDING current
    data['yx_spread_time_std_4hours'] = (
        data['yx_spread'].rolling(window=1410, min_periods=1).std().fillna(0) + std_reg_const
    )
    data['yx_spread_time_zscore_4hours'] = data['yx_spread_time_mean_4hours'] / data['yx_spread_time_std_4hours']
    
    # Spread lags (excluding current)
    data['yx_spread_time_mean_10min_lag_1hour'] = data['yx_spread_time_mean_10min'].shift(360)
    data['yx_spread_time_mean_2hours_lag_20min'] = data['yx_spread_time_mean_2hours'].shift(120)
    
    # Spread EWMA INCLUDING current
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'yx_spread_dayly_ewma_10min'] = (
            day_data['yx_spread'].ewm(halflife=60, adjust=False).mean()
        )
        data.loc[df_mask, 'yx_spread_dayly_ewma_1hour'] = (
            day_data['yx_spread'].ewm(halflife=360, adjust=False).mean()
        )
    
    data['yx_spread_dayly_ewma_10min'] = data['yx_spread'] - data['yx_spread_dayly_ewma_10min']
    data['yx_spread_dayly_ewma_1hour'] = data['yx_spread'] - data['yx_spread_dayly_ewma_1hour']
    data['yx_spread_ewma_prodpair_1hour_10min'] = (
        data.yx_spread_dayly_ewma_1hour * data.yx_spread_dayly_ewma_10min)
    
    # Clean up temporary columns
    temp_cols = [
        'xy_garmonic_dayly_ewma_1hour', 'xy_garmonic_dayly_ewma_2hours',
        'xy_square_time_mean_10min', 'xy_square_time_std_10min',
        'yx_spread_dayly_ewma_10min', 'yx_spread_dayly_ewma_1hour',
        'yprice_time_std_1hour', 'yprice_time_std_2hours',
        'yx_spread_time_std_4hours', 'xy_relation_time_std_1hour',
        'xy_relation_time_std_2hours'
    ]
    
    for col in temp_cols:
        if col in data.columns:
            data.drop(col, axis=1, inplace=True)
    
    print(f"Feature engineering completed (NO DATA LEAKAGE - {'TRAIN' if is_train else 'TEST'})!")
    return data

# %% Data Initialization
def init_data_single(fname):
    """Initialize and preprocess a single dataset"""
    print(f'Loading file: {fname}...')
    data = pd.read_csv(fname)
    
    data['xprice'] -= 127  # WARNING: Domain-specific adjustment
    data['yprice'] -= 146  # WARNING: Domain-specific adjustment
    
    # Create derived price features
    data['yx_spread'] = data.yprice - data.xprice
    data['yx_relation'] = data.yprice / data.xprice
    data['xy_relation'] = data.xprice / data.yprice
    data['xy_square'] = np.sqrt(data.xprice ** 2 + data.yprice ** 2) / 2
    data['xy_geom'] = np.sqrt(data.xprice * data.yprice)
    data['xy_garmonic'] = 2 / (1 / data.xprice + 1 / data.yprice)
    
    # Process timestamps
    data['timestamp'] = data['timestamp'] // 1000
    data['timestamp'] = data['timestamp'].apply(lambda stamp: datetime.fromtimestamp(stamp))
    data['timestamp'] = data['timestamp'] - pd.Timedelta(hours=1)
    data.index = data['timestamp']
    
    # Add time-based features
    data['weekday'] = data.timestamp.dt.weekday
    data['is_end_of_week'] = (data.timestamp.dt.weekday >= 2).astype(int)
    
    data['day'] = (data.timestamp.dt.date - data.timestamp.dt.date.min()).apply(lambda x: int(x.days))
    day_close_time = data.day.map(data.groupby('day').timestamp.max())
    data['periods_before_closing'] = (day_close_time - data.timestamp).apply(lambda x: x.seconds // 10)
    day_open_time = data.day.map(data.groupby('day').timestamp.min())
    data['periods_after_opening'] = (data.timestamp - day_open_time).apply(lambda x: x.seconds // 10)
    
    return data

def selected_features_extractor_separate(train_data_path, test_data_path):
    """
    Extract features separately for train and test to avoid leakage.
    This is the SAFE approach - features are computed independently.
    
    Parameters:
    train_data_path: Path to training data (can be None if only processing test)
    test_data_path: Path to test data (can be None if only processing train)
    """
    train = None
    test = None
    
    # Process training data if provided
    if train_data_path is not None:
        train_data = init_data_single(train_data_path)
        train_data = create_all_features(train_data, is_train=True)
        
        usecols = selected_cols + ['returns', 'periods_before_closing']
        train = train_data[usecols].iloc[droprows:].copy()
    
    # Process test data if provided
    if test_data_path is not None:
        test_data = init_data_single(test_data_path)
        test_data = create_all_features(test_data, is_train=False)
        
        # Test might not have 'returns' column
        test_cols = selected_cols + ['periods_before_closing']
        if 'returns' in test_data.columns:
            test_cols.append('returns')
        test = test_data[test_cols].copy()
    
    return train, test

# %% Model Training Functions
def normalize_train(df):
    """Normalize training data"""
    extended_cols = selected_cols + ['returns', 'periods_before_closing']
    norm_train = df[extended_cols].reset_index(drop=True).copy()
    norm_mean = norm_train[selected_cols].mean()
    norm_std = norm_train[selected_cols].std() + normalization_std_reg
    norm_train.loc[:,selected_cols] = (norm_train[selected_cols] - norm_mean) / norm_std
    return norm_train

def normalize_train_test(train, test):
    """Normalize train and test data using training statistics"""
    train_extended_cols = selected_cols + ['returns', 'periods_before_closing']
    test_extended_cols = selected_cols + ['periods_before_closing']
    
    norm_train = train[train_extended_cols].reset_index(drop=True).copy()
    norm_test = test[test_extended_cols].reset_index(drop=True).copy()
    
    norm_mean = norm_train[selected_cols].mean()
    norm_std = norm_train[selected_cols].std() + normalization_std_reg
    
    norm_train.loc[:,selected_cols] = (norm_train[selected_cols] - norm_mean) / norm_std
    norm_test.loc[:,selected_cols] = (norm_test[selected_cols] - norm_mean) / norm_std
    return norm_train, norm_test

# %% Model Estimation Function
def modelEstimate(train_data_path):
    """
    Train the model on the training data.
    
    Parameters:
    train_data_path (str): Path to the training CSV file
    
    Returns:
    dict: Dictionary containing the trained model and normalization parameters
    """
    print('--------->Start Estimation....')
    
    # Extract features (train only, no test contamination)
    train, _ = selected_features_extractor_separate(train_data_path, None)
    
    # Remove any rows with NaN values
    train = train.dropna()
    
    # Normalize training data
    print('----->Normalization....')
    norm_train = normalize_train(train)
    
    # Calculate normalization parameters for later use
    norm_mean = train[selected_cols].mean()
    norm_std = train[selected_cols].std() + normalization_std_reg
    
    # Validate model
    print('----->Validation....')
    model = Ridge(alpha=ridge_alpha)
    print(validate_model_by_pentate(model, norm_train, selected_cols, 0))
    
    # Fit final model
    print('----->Fitting....')
    model = Ridge(alpha=ridge_alpha)
    model.fit(norm_train[selected_cols], norm_train.returns)
    
    # Print feature importances
    print('----->Calculation feature importances...')
    print_importances(model, selected_cols)
    
    # Return model and parameters
    return {
        'model': model,
        'norm_mean': norm_mean,
        'norm_std': norm_std
    }

# %% Model Forecast Function
def modelForecast(test_data_path, model_params, train_data_path=None):
    """
    Make predictions on test data using the trained model.
    
    Parameters:
    test_data_path (str): Path to the test CSV file
    model_params (dict): Dictionary containing the trained model and normalization parameters
    train_data_path (str): Not used in this version (features computed separately)
    
    Returns:
    dict: Dictionary containing predictions and evaluation metrics (if test data has returns)
    """
    print('--------->Starting Forecasting....')
    
    # Extract features for test only (no train contamination)
    _, test = selected_features_extractor_separate(None, test_data_path)
    
    # Remove any rows with NaN values
    test = test.dropna()
    
    # Normalize test data using training statistics
    print('----->Normalization....')
    test_extended_cols = selected_cols + ['periods_before_closing']
    
    # Check if test data has returns column for evaluation
    has_returns = 'returns' in test.columns
    if has_returns:
        test_extended_cols.append('returns')
    
    norm_test = test[test_extended_cols].reset_index(drop=True).copy()
    norm_test.loc[:,selected_cols] = (norm_test[selected_cols] - model_params['norm_mean']) / model_params['norm_std']
    
    # Make predictions
    print('----->Prediction....')
    predicted = model_params['model'].predict(norm_test[selected_cols])
    
    # Set predictions to 0 at closing periods
    predicted[norm_test.periods_before_closing == 0] = 0
    
    # Calculate evaluation metrics if test data has returns
    results = {'predictions': predicted}
    
    if has_returns:
        print('----->Evaluation....')
        # Calculate MSE
        mse = mean_squared_error(norm_test.returns, predicted)
        # Calculate R-squared
        r2 = rsquared(norm_test.returns, predicted)
        
        results['mse'] = mse
        results['r2'] = r2
        
        print(f'Test MSE: {mse:.6f}')
        print(f'Test R-squared: {r2:.6f}')
        print(f'Test R-squared (x100): {r2*100:.2f}')
    else:
        print('No returns column in test data - evaluation metrics not calculated')
    
    return results

# %% Example Usage
if __name__ == "__main__":
    # Train the model
    train_file_path = '/Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/train.csv'  # Replace with your actual file path
    model_params = modelEstimate(train_file_path)
    
    # Save model for later use
    with open('model_params.pkl', 'wb') as f:
        pickle.dump(model_params, f)
    print("Model saved to model_params.pkl")
    
    # Make predictions on test data
    test_file_path = '/Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/test.csv'  # Replace with your actual file path
    
    # Load model if needed
    # with open('model_params.pkl', 'rb') as f:
    #     model_params = pickle.load(f)
    
    results = modelForecast(test_file_path, model_params)
    
    # Extract predictions
    predictions = results['predictions']
    
    # Display results
    print(f"\nFirst 10 predictions: {predictions[:10]}")
    print(f"Total predictions: {len(predictions)}")
    
    # Display evaluation metrics if available
    if 'mse' in results:
        print(f"\nEvaluation Metrics:")
        print(f"MSE: {results['mse']:.6f}")
        print(f"R-squared: {results['r2']:.6f}")
        print(f"R-squared (x100): {results['r2']*100:.2f}")
    
    # Save predictions to CSV
    predictions_df = pd.DataFrame({'predictions': predictions})
    predictions_df.to_csv('predictions.csv', index=False)
    print("\nPredictions saved to predictions.csv")

--------->Start Estimation....
Loading file: /Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/train.csv...
Creating all features (NO LEAKAGE - TRAIN)...
Creating opening/closing difference features...
Creating log features...
Creating expanding mean difference...
Creating time-based rolling features (TRULY FIXED)...
Creating intraday EWMA features...
Creating lag features...
Creating RSI features...
Creating EWMA on time means...
Creating XY combined features...
Creating YX spread features...
Feature engineering completed (NO DATA LEAKAGE - TRAIN)!
----->Normalization....
----->Validation....
     train_50_percent  train_60_percent  train_70_percent  train_80_percent  \
mse          0.031586          0.021226          0.017438          0.017618   
r2           0.507232          1.332249          0.024151          2.788663   

     train_90_percent  min_stats  max_stats       avg  
mse          0.020447   0.017438   0.031586  0.022477  
r2           0.711753  

# All features (worse performance) (exlcuding xy_relation_time_std_1hour and xy_relation_time_std_2hours as they are no longer calculated)

In [49]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from copy import copy
import scipy
import pickle

# %% Configuration
# Configuration parameters
droprows = 7050
std_reg_const = 0.1
normalization_std_reg = 0.0001
ridge_alpha = 1333

selected_cols = [
    
    # Time-based features
    'is_end_of_week',
    'weekday',
    
    # Opening/Closing difference features
    'xdiff_from_closing',
    'xdiff_from_opening',
    'ydiff_from_closing',
    
    # X-price features
    'xlog',
    'xlog_dayly_ewma_10min',
    'xprice_dayly_ewma_4hour',
    'xprice_lag_4hours',
    'xprice_time_mean_4hours',
    'xprice_time_mean_1hour_lag_4hours',
    'xprice_time_mean_1min',
    'xprice_time_mean_10min_dayly_ewma_10min',
    'xprice_time_mean_1min_rsi_1min',
    
    # Y-price features
    'ylog_dayly_ewma_1hour',
    'yprice_dayly_ewma_10min',
    'yprice_lag_4hours',
    'yprice_ewma_difpair_10min_4hour',
    'yprice_expanding_mean_diff', # FIXED: renamed from yprice_full_history_diff
    'yprice_time_mean_1hour',
    'yprice_time_mean_1hour_lag_4hours',
    'yprice_time_mean_1hour_lag_1workweek',
    'yprice_time_mean_10min',
    'yprice_time_mean_10min_dayly_ewma_4hour',
    'yprice_time_mean_10min_lag_10min',
    'yprice_time_mean_10min_rsi_1hour',
    'yprice_time_mean_2hours',
    'yprice_time_zscore_1hour',
    'yprice_time_zscore_2hours',
    
    # XY combined features
    'xy_garmonic_ewma_prodpair_2hours_1hour',
    'xy_garmonic_time_std_4hours',
    'xy_geom_time_mean_1hour_lag_20min',
    'xy_geom_time_mean_1min',
    'xy_geom_time_mean_10min_dayly_ewma_1min',
    'xy_geom_time_mean_2hours_dayly_ewma_20min',
    'xy_square_time_zscore_10min',
    
    # YX spread features
    'yx_spread_ewma_prodpair_1hour_10min',
    'yx_spread_time_mean_10min_lag_1hour',
    'yx_spread_time_mean_2hours_lag_20min',
    'yx_spread_time_zscore_4hours',
]


# %% Helper Functions
def print_importances(model, selected_cols):
    """Print feature importances sorted by absolute weight"""
    weigts_sum = sum(map(abs, model.coef_))
    for name, weight in sorted(zip(selected_cols, model.coef_), key=lambda x: -abs(x[1])):
        percent_weight = abs(weight) / weigts_sum
        print('{:40} {:.2%} {:15.2}'.format(name, percent_weight, weight))

def rsquared(x, y):
    """Return R^2 where x and y are array-like."""
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

def rsiFunc(prices, n=14):
    """Calculate RSI (Relative Strength Index)"""
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1]
        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n
        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)
    return rsi

# %% Time Series Validation Functions
def time_split(data, valid_ratio, test_ratio):
    """Split time series data into train, validation, and test sets"""
    n_valid = max(1, int(data.shape[0] * valid_ratio))
    n_test = max(1, int(data.shape[0] * test_ratio))
    n_train = data.shape[0] - n_valid - n_test
    
    train = data.iloc[:n_train].reset_index(drop=True).copy()
    valid = data.iloc[n_train:-n_test].reset_index(drop=True).copy()
    test = data.iloc[-n_test:].reset_index(drop=True).copy()
    merged_test = pd.concat([valid, test], ignore_index=True)
    return train, valid, test

def validate_model_by_pentate(model, source_data, base_cols, droprows=0):
    """Validate model using 5-fold time series cross-validation"""
    df = source_data.copy()
    selected_cols = base_cols.copy()
    helper_cols = list(set(selected_cols + ['periods_before_closing', 'returns']))
    metrics_dict = {}
    
    for step in range(5, 10):
        n_train = int(df.shape[0] * step // 10)
        n_test = int(df.shape[0] * (step + 1) // 10)
        train = df.iloc[:n_train].reset_index(drop=True).copy()
        test = df.iloc[n_train:n_test].reset_index(drop=True).copy()
        train.drop(np.arange(droprows), inplace=True)
        train.dropna(inplace=True)

        model.fit(train[selected_cols], train.returns)
        predicted = model.predict(test[selected_cols])
        predicted[test.periods_before_closing == 0] = 0

        current_mse = mean_squared_error(test.returns, predicted)
        current_r2 = rsquared(test.returns, predicted) * 100
        metrics_dict['train_{}_percent'.format(step * 10)] = {
            'mse': current_mse,
            'r2': current_r2
        }
    
    report = pd.DataFrame(metrics_dict)
    report['min_stats'] = report.iloc[:,:5].min(1).astype(np.float32)
    report['max_stats'] = report.iloc[:,:5].max(1).astype(np.float32)
    report['avg'] = report.mean(1).astype(np.float32)
    return report

# %% TRULY FIXED Feature Engineering Function
def create_all_features(data, is_train=True):
    """
    TRULY FIXED: Feature engineering with NO data leakage.
    All features use strictly backward-looking rolling windows.
    Current observation at time t IS included (since we predict t+60).
    No resample+ffill pattern that causes intra-bin leakage.
    """
    print(f"Creating all features (NO LEAKAGE - {'TRAIN' if is_train else 'TEST'})...")
    
    # Pre-calculate commonly used values
    days = data.day.unique()
    
    # Opening/Closing differences (using previous day's close)
    print("Creating opening/closing difference features...")
    close_price_per_day_y = data.groupby('day').timestamp.max().shift(1).map(
        data[['timestamp', 'yprice']].set_index('timestamp').yprice)
    data['ydiff_from_closing'] = (data.yprice - data.day.map(close_price_per_day_y)).fillna(0)
    
    close_price_per_day_x = data.groupby('day').timestamp.max().shift(1).map(
        data[['timestamp', 'xprice']].set_index('timestamp').xprice)
    data['xdiff_from_closing'] = (data.xprice - data.day.map(close_price_per_day_x)).fillna(0)
    
    open_price_per_day_x = data.groupby('day').timestamp.min().map(
        data[['timestamp', 'xprice']].set_index('timestamp').xprice)
    data['xdiff_from_opening'] = data.xprice - data.day.map(open_price_per_day_x)
    
    # Log features
    print("Creating log features...")
    data['xlog'] = data.xprice.apply(np.log1p)
    data['ylog'] = data.yprice.apply(np.log1p)
    
    # Expanding mean that only uses past data (including current)
    print("Creating expanding mean difference...")
    data['yprice_expanding_mean_diff'] = data['yprice'] - data['yprice'].expanding(min_periods=1).mean()
    
    # FIXED: Time-based rolling features with backward-looking windows INCLUDING current
    print("Creating time-based rolling features (TRULY FIXED)...")
    time_windows = {
        6: '1min', 60: '10min', 360: '1hour', 720: '2hours', 1410: '4hours', 2820: '1workweek'
    }
    
    # Create rolling means with backward-looking windows INCLUDING current observation
    for window, name in time_windows.items():
        if window in [6, 60, 360, 1410]:  # xprice windows
            colname = f'xprice_time_mean_{name}'
            # Rolling includes current observation (no shift)
            data[colname] = data['xprice'] - data['xprice'].rolling(
                window=window, min_periods=1).mean()
        
        if window in [60, 360, 720]:  # yprice windows
            colname = f'yprice_time_mean_{name}'
            # Rolling includes current observation (no shift)
            data[colname] = data['yprice'] - data['yprice'].rolling(
                window=window, min_periods=1).mean()
    
    # Create rolling std for yprice with backward-looking windows INCLUDING current
    for window in [360, 720]:
        name = time_windows[window]
        colname = f'yprice_time_std_{name}'
        # Rolling includes current observation (no shift)
        data[colname] = data['yprice'].rolling(
            window=window, min_periods=1).std().fillna(0) + std_reg_const
    
    # Z-scores for yprice
    data['yprice_time_zscore_1hour'] = data.yprice_time_mean_1hour / data.yprice_time_std_1hour
    data['yprice_time_zscore_2hours'] = data.yprice_time_mean_2hours / data.yprice_time_std_2hours
    
    # Intraday EWMA features (backward-looking INCLUDING current)
    print("Creating intraday EWMA features...")
    ewma_configs = [
        ('xprice', [24], ['4hour']),
        ('xlog', [60], ['10min']),
        ('ylog', [360], ['1hour']),
        ('yprice', [24, 60], ['4hour', '10min'])
    ]
    
    for col, windows, names in ewma_configs:
        for day in days:
            df_mask = (data.day == day)
            day_data = data.loc[df_mask].copy()
            
            for window, name in zip(windows, names):
                colname = f'{col}_dayly_ewma_{name}'
                # EWMA includes current observation (no shift)
                ewm = day_data[col].ewm(halflife=window, adjust=False).mean()
                
                if col in ['xprice', 'yprice']:
                    data.loc[df_mask, colname] = day_data[col] - ewm
                elif col in ['xlog', 'ylog']:
                    data.loc[df_mask, colname] = day_data[col.replace('log', 'price')] - ewm
    
    # EWMA difference pair
    data['yprice_ewma_difpair_10min_4hour'] = data.yprice_dayly_ewma_10min - data.yprice_dayly_ewma_4hour
    
    # Lag features (inherently backward-looking, excludes current)
    print("Creating lag features...")
    lag_configs = [
        ('xprice', 1410, '4hours'),
        ('yprice', 1410, '4hours'),
        ('xprice_time_mean_1hour', 1410, '4hours'),
        ('yprice_time_mean_1hour', 1410, '4hours'),
        ('yprice_time_mean_1hour', 2820, '1workweek'),
        ('yprice_time_mean_10min', 60, '10min')
    ]
    
    for col, lag, name in lag_configs:
        data[f'{col}_lag_{name}'] = data[col].shift(lag)
    
    # RSI features (calculated on backward-looking data including current)
    print("Creating RSI features...")
    # RSI needs some history, so we calculate it on the rolling means
    if 'xprice_time_mean_1min' in data.columns:
        data['xprice_time_mean_1min_rsi_1min'] = rsiFunc(data['xprice_time_mean_1min'].fillna(0).values, 6)
    data['yprice_time_mean_10min_rsi_1hour'] = rsiFunc(data['yprice_time_mean_10min'].fillna(0).values, 360)
    
    # Additional EWMA on time means
    print("Creating EWMA on time means...")
    ewma_on_means = [
        ('xprice_time_mean_10min', 60, '10min'),
        ('yprice_time_mean_10min', 24, '4hour')
    ]
    
    for col, window, name in ewma_on_means:
        for day in days:
            df_mask = (data.day == day)
            day_data = data.loc[df_mask].copy()
            colname = f'{col}_dayly_ewma_{name}'
            # EWMA includes current observation (no shift)
            ewm = day_data[col].ewm(halflife=window, adjust=False).mean()
            data.loc[df_mask, colname] = ewm
    
    # XY Combined features with backward-looking windows INCLUDING current
    print("Creating XY combined features...")
    
    # XY Harmonic std - use rolling window INCLUDING current
    data['xy_garmonic_time_std_4hours'] = (
        data['xy_garmonic'].rolling(window=1410, min_periods=1).std().fillna(0) + std_reg_const
    )
    
    # Harmonic EWMA
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'xy_garmonic_dayly_ewma_1hour'] = (
            day_data['xy_garmonic'].ewm(halflife=360, adjust=False).mean()
        )
        data.loc[df_mask, 'xy_garmonic_dayly_ewma_2hours'] = (
            day_data['xy_garmonic'].ewm(halflife=720, adjust=False).mean()
        )
    
    data['xy_garmonic_dayly_ewma_1hour'] = data['xy_garmonic'] - data['xy_garmonic_dayly_ewma_1hour']
    data['xy_garmonic_dayly_ewma_2hours'] = data['xy_garmonic'] - data['xy_garmonic_dayly_ewma_2hours']
    data['xy_garmonic_ewma_prodpair_2hours_1hour'] = (
        data.xy_garmonic_dayly_ewma_2hours * data.xy_garmonic_dayly_ewma_1hour)
    
    # XY Geometric features with rolling windows INCLUDING current
    for window, name in [(6, '1min'), (60, '10min'), (360, '1hour'), (720, '2hours')]:
        colname = f'xy_geom_time_mean_{name}'
        # Rolling includes current observation (no shift)
        data[colname] = data['xy_geom'] - data['xy_geom'].rolling(
            window=window, min_periods=1).mean()
    
    # Geometric lags and EWMA
    data['xy_geom_time_mean_1hour_lag_20min'] = data['xy_geom_time_mean_1hour'].shift(120)
    
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'xy_geom_time_mean_10min_dayly_ewma_1min'] = (
            day_data['xy_geom_time_mean_10min'].ewm(halflife=6, adjust=False).mean()
        )
        data.loc[df_mask, 'xy_geom_time_mean_2hours_dayly_ewma_20min'] = (
            day_data['xy_geom_time_mean_2hours'].ewm(halflife=120, adjust=False).mean()
        )
    
    # XY Relation std with rolling windows INCLUDING current
    for window, name in [(360, '1hour'), (720, '2hours')]:
        colname = f'xy_relation_time_std_{name}'
        data[colname] = (
            data['xy_relation'].rolling(window=window, min_periods=1).std().fillna(0) + std_reg_const
        )
    
    # XY Square zscore with rolling windows INCLUDING current
    data['xy_square_time_mean_10min'] = (
        data['xy_square'] - data['xy_square'].rolling(window=60, min_periods=1).mean()
    )
    data['xy_square_time_std_10min'] = (
        data['xy_square'].rolling(window=60, min_periods=1).std().fillna(0) + std_reg_const
    )
    data['xy_square_time_zscore_10min'] = data['xy_square_time_mean_10min'] / data['xy_square_time_std_10min']
    
    # YX Spread features with rolling windows INCLUDING current
    print("Creating YX spread features...")
    for window, name in [(60, '10min'), (720, '2hours'), (1410, '4hours')]:
        colname = f'yx_spread_time_mean_{name}'
        data[colname] = (
            data['yx_spread'] - data['yx_spread'].rolling(window=window, min_periods=1).mean()
        )
    
    # Spread std and zscore with rolling windows INCLUDING current
    data['yx_spread_time_std_4hours'] = (
        data['yx_spread'].rolling(window=1410, min_periods=1).std().fillna(0) + std_reg_const
    )
    data['yx_spread_time_zscore_4hours'] = data['yx_spread_time_mean_4hours'] / data['yx_spread_time_std_4hours']
    
    # Spread lags (excluding current)
    data['yx_spread_time_mean_10min_lag_1hour'] = data['yx_spread_time_mean_10min'].shift(360)
    data['yx_spread_time_mean_2hours_lag_20min'] = data['yx_spread_time_mean_2hours'].shift(120)
    
    # Spread EWMA INCLUDING current
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'yx_spread_dayly_ewma_10min'] = (
            day_data['yx_spread'].ewm(halflife=60, adjust=False).mean()
        )
        data.loc[df_mask, 'yx_spread_dayly_ewma_1hour'] = (
            day_data['yx_spread'].ewm(halflife=360, adjust=False).mean()
        )
    
    data['yx_spread_dayly_ewma_10min'] = data['yx_spread'] - data['yx_spread_dayly_ewma_10min']
    data['yx_spread_dayly_ewma_1hour'] = data['yx_spread'] - data['yx_spread_dayly_ewma_1hour']
    data['yx_spread_ewma_prodpair_1hour_10min'] = (
        data.yx_spread_dayly_ewma_1hour * data.yx_spread_dayly_ewma_10min)
    
    # Clean up temporary columns
    temp_cols = [
        'xy_garmonic_dayly_ewma_1hour', 'xy_garmonic_dayly_ewma_2hours',
        'xy_square_time_mean_10min', 'xy_square_time_std_10min',
        'yx_spread_dayly_ewma_10min', 'yx_spread_dayly_ewma_1hour',
        'yprice_time_std_1hour', 'yprice_time_std_2hours',
        'yx_spread_time_std_4hours', 'xy_relation_time_std_1hour',
        'xy_relation_time_std_2hours'
    ]
    
    for col in temp_cols:
        if col in data.columns:
            data.drop(col, axis=1, inplace=True)
    
    print(f"Feature engineering completed (NO DATA LEAKAGE - {'TRAIN' if is_train else 'TEST'})!")
    return data

# %% Data Initialization
def init_data_single(fname):
    """Initialize and preprocess a single dataset"""
    print(f'Loading file: {fname}...')
    data = pd.read_csv(fname)
    
    data['xprice'] -= 127  # WARNING: Domain-specific adjustment
    data['yprice'] -= 146  # WARNING: Domain-specific adjustment
    
    # Create derived price features
    data['yx_spread'] = data.yprice - data.xprice
    data['yx_relation'] = data.yprice / data.xprice
    data['xy_relation'] = data.xprice / data.yprice
    data['xy_square'] = np.sqrt(data.xprice ** 2 + data.yprice ** 2) / 2
    data['xy_geom'] = np.sqrt(data.xprice * data.yprice)
    data['xy_garmonic'] = 2 / (1 / data.xprice + 1 / data.yprice)
    
    # Process timestamps
    data['timestamp'] = data['timestamp'] // 1000
    data['timestamp'] = data['timestamp'].apply(lambda stamp: datetime.fromtimestamp(stamp))
    data['timestamp'] = data['timestamp'] - pd.Timedelta(hours=1)
    data.index = data['timestamp']
    
    # Add time-based features
    data['weekday'] = data.timestamp.dt.weekday
    data['is_end_of_week'] = (data.timestamp.dt.weekday >= 2).astype(int)
    
    data['day'] = (data.timestamp.dt.date - data.timestamp.dt.date.min()).apply(lambda x: int(x.days))
    day_close_time = data.day.map(data.groupby('day').timestamp.max())
    data['periods_before_closing'] = (day_close_time - data.timestamp).apply(lambda x: x.seconds // 10)
    day_open_time = data.day.map(data.groupby('day').timestamp.min())
    data['periods_after_opening'] = (data.timestamp - day_open_time).apply(lambda x: x.seconds // 10)
    
    return data

def selected_features_extractor_separate(train_data_path, test_data_path):
    """
    Extract features separately for train and test to avoid leakage.
    This is the SAFE approach - features are computed independently.
    
    Parameters:
    train_data_path: Path to training data (can be None if only processing test)
    test_data_path: Path to test data (can be None if only processing train)
    """
    train = None
    test = None
    
    # Process training data if provided
    if train_data_path is not None:
        train_data = init_data_single(train_data_path)
        train_data = create_all_features(train_data, is_train=True)
        
        usecols = selected_cols + ['returns', 'periods_before_closing']
        train = train_data[usecols].iloc[droprows:].copy()
    
    # Process test data if provided
    if test_data_path is not None:
        test_data = init_data_single(test_data_path)
        test_data = create_all_features(test_data, is_train=False)
        
        # Test might not have 'returns' column
        test_cols = selected_cols + ['periods_before_closing']
        if 'returns' in test_data.columns:
            test_cols.append('returns')
        test = test_data[test_cols].copy()
    
    return train, test

# %% Model Training Functions
def normalize_train(df):
    """Normalize training data"""
    extended_cols = selected_cols + ['returns', 'periods_before_closing']
    norm_train = df[extended_cols].reset_index(drop=True).copy()
    norm_mean = norm_train[selected_cols].mean()
    norm_std = norm_train[selected_cols].std() + normalization_std_reg
    norm_train.loc[:,selected_cols] = (norm_train[selected_cols] - norm_mean) / norm_std
    return norm_train

def normalize_train_test(train, test):
    """Normalize train and test data using training statistics"""
    train_extended_cols = selected_cols + ['returns', 'periods_before_closing']
    test_extended_cols = selected_cols + ['periods_before_closing']
    
    norm_train = train[train_extended_cols].reset_index(drop=True).copy()
    norm_test = test[test_extended_cols].reset_index(drop=True).copy()
    
    norm_mean = norm_train[selected_cols].mean()
    norm_std = norm_train[selected_cols].std() + normalization_std_reg
    
    norm_train.loc[:,selected_cols] = (norm_train[selected_cols] - norm_mean) / norm_std
    norm_test.loc[:,selected_cols] = (norm_test[selected_cols] - norm_mean) / norm_std
    return norm_train, norm_test

# %% Model Estimation Function
def modelEstimate(train_data_path):
    """
    Train the model on the training data.
    
    Parameters:
    train_data_path (str): Path to the training CSV file
    
    Returns:
    dict: Dictionary containing the trained model and normalization parameters
    """
    print('--------->Start Estimation....')
    
    # Extract features (train only, no test contamination)
    train, _ = selected_features_extractor_separate(train_data_path, None)
    
    # Remove any rows with NaN values
    train = train.dropna()
    
    # Normalize training data
    print('----->Normalization....')
    norm_train = normalize_train(train)
    
    # Calculate normalization parameters for later use
    norm_mean = train[selected_cols].mean()
    norm_std = train[selected_cols].std() + normalization_std_reg
    
    # Validate model
    print('----->Validation....')
    model = Ridge(alpha=ridge_alpha)
    print(validate_model_by_pentate(model, norm_train, selected_cols, 0))
    
    # Fit final model
    print('----->Fitting....')
    model = Ridge(alpha=ridge_alpha)
    model.fit(norm_train[selected_cols], norm_train.returns)
    
    # Print feature importances
    print('----->Calculation feature importances...')
    print_importances(model, selected_cols)
    
    # Return model and parameters
    return {
        'model': model,
        'norm_mean': norm_mean,
        'norm_std': norm_std
    }

# %% Model Forecast Function
def modelForecast(test_data_path, model_params, train_data_path=None):
    """
    Make predictions on test data using the trained model.
    
    Parameters:
    test_data_path (str): Path to the test CSV file
    model_params (dict): Dictionary containing the trained model and normalization parameters
    train_data_path (str): Not used in this version (features computed separately)
    
    Returns:
    dict: Dictionary containing predictions and evaluation metrics (if test data has returns)
    """
    print('--------->Starting Forecasting....')
    
    # Extract features for test only (no train contamination)
    _, test = selected_features_extractor_separate(None, test_data_path)
    
    # Remove any rows with NaN values
    test = test.dropna()
    
    # Normalize test data using training statistics
    print('----->Normalization....')
    test_extended_cols = selected_cols + ['periods_before_closing']
    
    # Check if test data has returns column for evaluation
    has_returns = 'returns' in test.columns
    if has_returns:
        test_extended_cols.append('returns')
    
    norm_test = test[test_extended_cols].reset_index(drop=True).copy()
    norm_test.loc[:,selected_cols] = (norm_test[selected_cols] - model_params['norm_mean']) / model_params['norm_std']
    
    # Make predictions
    print('----->Prediction....')
    predicted = model_params['model'].predict(norm_test[selected_cols])
    
    # Set predictions to 0 at closing periods
    predicted[norm_test.periods_before_closing == 0] = 0
    
    # Calculate evaluation metrics if test data has returns
    results = {'predictions': predicted}
    
    if has_returns:
        print('----->Evaluation....')
        # Calculate MSE
        mse = mean_squared_error(norm_test.returns, predicted)
        # Calculate R-squared
        r2 = rsquared(norm_test.returns, predicted)
        
        results['mse'] = mse
        results['r2'] = r2
        
        print(f'Test MSE: {mse:.6f}')
        print(f'Test R-squared: {r2:.6f}')
        print(f'Test R-squared (x100): {r2*100:.2f}')
    else:
        print('No returns column in test data - evaluation metrics not calculated')
    
    return results

# %% Example Usage
if __name__ == "__main__":
    # Train the model
    train_file_path = '/Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/train.csv'  # Replace with your actual file path
    model_params = modelEstimate(train_file_path)
    
    # Save model for later use
    with open('model_params.pkl', 'wb') as f:
        pickle.dump(model_params, f)
    print("Model saved to model_params.pkl")
    
    # Make predictions on test data
    test_file_path = '/Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/test.csv'  # Replace with your actual file path
    
    # Load model if needed
    # with open('model_params.pkl', 'rb') as f:
    #     model_params = pickle.load(f)
    
    results = modelForecast(test_file_path, model_params)
    
    # Extract predictions
    predictions = results['predictions']
    
    # Display results
    print(f"\nFirst 10 predictions: {predictions[:10]}")
    print(f"Total predictions: {len(predictions)}")
    
    # Display evaluation metrics if available
    if 'mse' in results:
        print(f"\nEvaluation Metrics:")
        print(f"MSE: {results['mse']:.6f}")
        print(f"R-squared: {results['r2']:.6f}")
        print(f"R-squared (x100): {results['r2']*100:.2f}")
    
    # Save predictions to CSV
    predictions_df = pd.DataFrame({'predictions': predictions})
    predictions_df.to_csv('predictions.csv', index=False)
    print("\nPredictions saved to predictions.csv")

--------->Start Estimation....
Loading file: /Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/train.csv...
Creating all features (NO LEAKAGE - TRAIN)...
Creating opening/closing difference features...
Creating log features...
Creating expanding mean difference...
Creating time-based rolling features (TRULY FIXED)...
Creating intraday EWMA features...
Creating lag features...
Creating RSI features...
Creating EWMA on time means...
Creating XY combined features...
Creating YX spread features...
Feature engineering completed (NO DATA LEAKAGE - TRAIN)!
----->Normalization....
----->Validation....
     train_50_percent  train_60_percent  train_70_percent  train_80_percent  \
mse          0.031237          0.021398          0.017521          0.017257   
r2           0.593954          0.844598          0.033450          3.592538   

     train_90_percent  min_stats  max_stats       avg  
mse          0.020141   0.017257   0.031237  0.022293  
r2           1.903203  

# XGBoost with all features 

In [77]:
import pandas as pd
import numpy as np
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from copy import copy
import scipy
import pickle

# %% Configuration
# Configuration parameters
droprows = 7050
std_reg_const = 0.1
normalization_std_reg = 0.0001

# XGBoost parameters
xgb_params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.05,
    'n_estimators': 500,
    'max_depth': 5,
    'min_child_weight': 6,
    'gamma': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'random_state': 42
}

selected_cols = [
    
    # Time-based features
    'is_end_of_week',
    'weekday',
    
    # Opening/Closing difference features
    'xdiff_from_closing',
    'xdiff_from_opening',
    'ydiff_from_closing',
    
    # X-price features
    'xlog',
    'xlog_dayly_ewma_10min',
    'xprice_dayly_ewma_4hour',
    'xprice_lag_4hours',
    'xprice_time_mean_4hours',
    'xprice_time_mean_1hour_lag_4hours',
    'xprice_time_mean_1min',
    'xprice_time_mean_10min_dayly_ewma_10min',
    'xprice_time_mean_1min_rsi_1min',
    
    # Y-price features
    'ylog_dayly_ewma_1hour',
    'yprice_dayly_ewma_10min',
    'yprice_lag_4hours',
    'yprice_ewma_difpair_10min_4hour',
    'yprice_expanding_mean_diff', # FIXED: renamed from yprice_full_history_diff
    'yprice_time_mean_1hour',
    'yprice_time_mean_1hour_lag_4hours',
    'yprice_time_mean_1hour_lag_1workweek',
    'yprice_time_mean_10min',
    'yprice_time_mean_10min_dayly_ewma_4hour',
    'yprice_time_mean_10min_lag_10min',
    'yprice_time_mean_10min_rsi_1hour',
    'yprice_time_mean_2hours',
    'yprice_time_zscore_1hour',
    'yprice_time_zscore_2hours',
    
    # XY combined features
    'xy_garmonic_ewma_prodpair_2hours_1hour',
    'xy_garmonic_time_std_4hours',
    'xy_geom_time_mean_1hour_lag_20min',
    'xy_geom_time_mean_1min',
    'xy_geom_time_mean_10min_dayly_ewma_1min',
    'xy_geom_time_mean_2hours_dayly_ewma_20min',
    'xy_square_time_zscore_10min',
    
    # YX spread features
    'yx_spread_ewma_prodpair_1hour_10min',
    'yx_spread_time_mean_10min_lag_1hour',
    'yx_spread_time_mean_2hours_lag_20min',
    'yx_spread_time_zscore_4hours',
]


# %% Helper Functions
def print_importances_xgb(model, selected_cols):
    """Print feature importances for XGBoost model sorted by importance"""
    feature_importance = model.feature_importances_
    importance_dict = dict(zip(selected_cols, feature_importance))
    
    # Sort by importance
    sorted_features = sorted(importance_dict.items(), key=lambda x: -x[1])
    
    # Calculate percentage weights
    total_importance = sum(feature_importance)
    
    print("\nFeature Importances (XGBoost):")
    print("-" * 70)
    for name, importance in sorted_features:
        percent_importance = importance / total_importance if total_importance > 0 else 0
        print('{:40} {:.2%} {:15.6f}'.format(name, percent_importance, importance))

def rsquared(x, y):
    """Return R^2 where x and y are array-like."""
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

def rsiFunc(prices, n=14):
    """Calculate RSI (Relative Strength Index)"""
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1]
        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n
        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)
    return rsi

# %% Time Series Validation Functions
def time_split(data, valid_ratio, test_ratio):
    """Split time series data into train, validation, and test sets"""
    n_valid = max(1, int(data.shape[0] * valid_ratio))
    n_test = max(1, int(data.shape[0] * test_ratio))
    n_train = data.shape[0] - n_valid - n_test
    
    train = data.iloc[:n_train].reset_index(drop=True).copy()
    valid = data.iloc[n_train:-n_test].reset_index(drop=True).copy()
    test = data.iloc[-n_test:].reset_index(drop=True).copy()
    merged_test = pd.concat([valid, test], ignore_index=True)
    return train, valid, test

def validate_model_by_pentate(model, source_data, base_cols, droprows=0):
    """Validate model using 5-fold time series cross-validation"""
    df = source_data.copy()
    selected_cols = base_cols.copy()
    helper_cols = list(set(selected_cols + ['periods_before_closing', 'returns']))
    metrics_dict = {}
    
    for step in range(5, 10):
        n_train = int(df.shape[0] * step // 10)
        n_test = int(df.shape[0] * (step + 1) // 10)
        train = df.iloc[:n_train].reset_index(drop=True).copy()
        test = df.iloc[n_train:n_test].reset_index(drop=True).copy()
        train.drop(np.arange(droprows), inplace=True)
        train.dropna(inplace=True)

        # For XGBoost, we need to create a new model instance for each fold
        fold_model = XGBRegressor(**xgb_params)
        fold_model.fit(train[selected_cols], train.returns)
        predicted = fold_model.predict(test[selected_cols])
        predicted[test.periods_before_closing == 0] = 0

        current_mse = mean_squared_error(test.returns, predicted)
        current_r2 = rsquared(test.returns, predicted) * 100
        metrics_dict['train_{}_percent'.format(step * 10)] = {
            'mse': current_mse,
            'r2': current_r2
        }
    
    report = pd.DataFrame(metrics_dict)
    report['min_stats'] = report.iloc[:,:5].min(1).astype(np.float32)
    report['max_stats'] = report.iloc[:,:5].max(1).astype(np.float32)
    report['avg'] = report.mean(1).astype(np.float32)
    return report

# %% TRULY FIXED Feature Engineering Function
def create_all_features(data, is_train=True):
    """
    TRULY FIXED: Feature engineering with NO data leakage.
    All features use strictly backward-looking rolling windows.
    Current observation at time t IS included (since we predict t+60).
    No resample+ffill pattern that causes intra-bin leakage.
    """
    print(f"Creating all features (NO LEAKAGE - {'TRAIN' if is_train else 'TEST'})...")
    
    # Pre-calculate commonly used values
    days = data.day.unique()
    
    # Opening/Closing differences (using previous day's close)
    print("Creating opening/closing difference features...")
    close_price_per_day_y = data.groupby('day').timestamp.max().shift(1).map(
        data[['timestamp', 'yprice']].set_index('timestamp').yprice)
    data['ydiff_from_closing'] = (data.yprice - data.day.map(close_price_per_day_y)).fillna(0)
    
    close_price_per_day_x = data.groupby('day').timestamp.max().shift(1).map(
        data[['timestamp', 'xprice']].set_index('timestamp').xprice)
    data['xdiff_from_closing'] = (data.xprice - data.day.map(close_price_per_day_x)).fillna(0)
    
    open_price_per_day_x = data.groupby('day').timestamp.min().map(
        data[['timestamp', 'xprice']].set_index('timestamp').xprice)
    data['xdiff_from_opening'] = data.xprice - data.day.map(open_price_per_day_x)
    
    # Log features
    print("Creating log features...")
    data['xlog'] = data.xprice.apply(np.log1p)
    data['ylog'] = data.yprice.apply(np.log1p)
    
    # Expanding mean that only uses past data (including current)
    print("Creating expanding mean difference...")
    data['yprice_expanding_mean_diff'] = data['yprice'] - data['yprice'].expanding(min_periods=1).mean()
    
    # FIXED: Time-based rolling features with backward-looking windows INCLUDING current
    print("Creating time-based rolling features (TRULY FIXED)...")
    time_windows = {
        6: '1min', 60: '10min', 360: '1hour', 720: '2hours', 1410: '4hours', 2820: '1workweek'
    }
    
    # Create rolling means with backward-looking windows INCLUDING current observation
    for window, name in time_windows.items():
        if window in [6, 60, 360, 1410]:  # xprice windows
            colname = f'xprice_time_mean_{name}'
            # Rolling includes current observation (no shift)
            data[colname] = data['xprice'] - data['xprice'].rolling(
                window=window, min_periods=1).mean()
        
        if window in [60, 360, 720]:  # yprice windows
            colname = f'yprice_time_mean_{name}'
            # Rolling includes current observation (no shift)
            data[colname] = data['yprice'] - data['yprice'].rolling(
                window=window, min_periods=1).mean()
    
    # Create rolling std for yprice with backward-looking windows INCLUDING current
    for window in [360, 720]:
        name = time_windows[window]
        colname = f'yprice_time_std_{name}'
        # Rolling includes current observation (no shift)
        data[colname] = data['yprice'].rolling(
            window=window, min_periods=1).std().fillna(0) + std_reg_const
    
    # Z-scores for yprice
    data['yprice_time_zscore_1hour'] = data.yprice_time_mean_1hour / data.yprice_time_std_1hour
    data['yprice_time_zscore_2hours'] = data.yprice_time_mean_2hours / data.yprice_time_std_2hours
    
    # Intraday EWMA features (backward-looking INCLUDING current)
    print("Creating intraday EWMA features...")
    ewma_configs = [
        ('xprice', [24], ['4hour']),
        ('xlog', [60], ['10min']),
        ('ylog', [360], ['1hour']),
        ('yprice', [24, 60], ['4hour', '10min'])
    ]
    
    for col, windows, names in ewma_configs:
        for day in days:
            df_mask = (data.day == day)
            day_data = data.loc[df_mask].copy()
            
            for window, name in zip(windows, names):
                colname = f'{col}_dayly_ewma_{name}'
                # EWMA includes current observation (no shift)
                ewm = day_data[col].ewm(halflife=window, adjust=False).mean()
                
                if col in ['xprice', 'yprice']:
                    data.loc[df_mask, colname] = day_data[col] - ewm
                elif col in ['xlog', 'ylog']:
                    data.loc[df_mask, colname] = day_data[col.replace('log', 'price')] - ewm
    
    # EWMA difference pair
    data['yprice_ewma_difpair_10min_4hour'] = data.yprice_dayly_ewma_10min - data.yprice_dayly_ewma_4hour
    
    # Lag features (inherently backward-looking, excludes current)
    print("Creating lag features...")
    lag_configs = [
        ('xprice', 1410, '4hours'),
        ('yprice', 1410, '4hours'),
        ('xprice_time_mean_1hour', 1410, '4hours'),
        ('yprice_time_mean_1hour', 1410, '4hours'),
        ('yprice_time_mean_1hour', 2820, '1workweek'),
        ('yprice_time_mean_10min', 60, '10min')
    ]
    
    for col, lag, name in lag_configs:
        data[f'{col}_lag_{name}'] = data[col].shift(lag)
    
    # RSI features (calculated on backward-looking data including current)
    print("Creating RSI features...")
    # RSI needs some history, so we calculate it on the rolling means
    if 'xprice_time_mean_1min' in data.columns:
        data['xprice_time_mean_1min_rsi_1min'] = rsiFunc(data['xprice_time_mean_1min'].fillna(0).values, 6)
    data['yprice_time_mean_10min_rsi_1hour'] = rsiFunc(data['yprice_time_mean_10min'].fillna(0).values, 360)
    
    # Additional EWMA on time means
    print("Creating EWMA on time means...")
    ewma_on_means = [
        ('xprice_time_mean_10min', 60, '10min'),
        ('yprice_time_mean_10min', 24, '4hour')
    ]
    
    for col, window, name in ewma_on_means:
        for day in days:
            df_mask = (data.day == day)
            day_data = data.loc[df_mask].copy()
            colname = f'{col}_dayly_ewma_{name}'
            # EWMA includes current observation (no shift)
            ewm = day_data[col].ewm(halflife=window, adjust=False).mean()
            data.loc[df_mask, colname] = ewm
    
    # XY Combined features with backward-looking windows INCLUDING current
    print("Creating XY combined features...")
    
    # XY Harmonic std - use rolling window INCLUDING current
    data['xy_garmonic_time_std_4hours'] = (
        data['xy_garmonic'].rolling(window=1410, min_periods=1).std().fillna(0) + std_reg_const
    )
    
    # Harmonic EWMA
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'xy_garmonic_dayly_ewma_1hour'] = (
            day_data['xy_garmonic'].ewm(halflife=360, adjust=False).mean()
        )
        data.loc[df_mask, 'xy_garmonic_dayly_ewma_2hours'] = (
            day_data['xy_garmonic'].ewm(halflife=720, adjust=False).mean()
        )
    
    data['xy_garmonic_dayly_ewma_1hour'] = data['xy_garmonic'] - data['xy_garmonic_dayly_ewma_1hour']
    data['xy_garmonic_dayly_ewma_2hours'] = data['xy_garmonic'] - data['xy_garmonic_dayly_ewma_2hours']
    data['xy_garmonic_ewma_prodpair_2hours_1hour'] = (
        data.xy_garmonic_dayly_ewma_2hours * data.xy_garmonic_dayly_ewma_1hour)
    
    # XY Geometric features with rolling windows INCLUDING current
    for window, name in [(6, '1min'), (60, '10min'), (360, '1hour'), (720, '2hours')]:
        colname = f'xy_geom_time_mean_{name}'
        # Rolling includes current observation (no shift)
        data[colname] = data['xy_geom'] - data['xy_geom'].rolling(
            window=window, min_periods=1).mean()
    
    # Geometric lags and EWMA
    data['xy_geom_time_mean_1hour_lag_20min'] = data['xy_geom_time_mean_1hour'].shift(120)
    
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'xy_geom_time_mean_10min_dayly_ewma_1min'] = (
            day_data['xy_geom_time_mean_10min'].ewm(halflife=6, adjust=False).mean()
        )
        data.loc[df_mask, 'xy_geom_time_mean_2hours_dayly_ewma_20min'] = (
            day_data['xy_geom_time_mean_2hours'].ewm(halflife=120, adjust=False).mean()
        )
    
    # XY Relation std with rolling windows INCLUDING current
    for window, name in [(360, '1hour'), (720, '2hours')]:
        colname = f'xy_relation_time_std_{name}'
        data[colname] = (
            data['xy_relation'].rolling(window=window, min_periods=1).std().fillna(0) + std_reg_const
        )
    
    # XY Square zscore with rolling windows INCLUDING current
    data['xy_square_time_mean_10min'] = (
        data['xy_square'] - data['xy_square'].rolling(window=60, min_periods=1).mean()
    )
    data['xy_square_time_std_10min'] = (
        data['xy_square'].rolling(window=60, min_periods=1).std().fillna(0) + std_reg_const
    )
    data['xy_square_time_zscore_10min'] = data['xy_square_time_mean_10min'] / data['xy_square_time_std_10min']
    
    # YX Spread features with rolling windows INCLUDING current
    print("Creating YX spread features...")
    for window, name in [(60, '10min'), (720, '2hours'), (1410, '4hours')]:
        colname = f'yx_spread_time_mean_{name}'
        data[colname] = (
            data['yx_spread'] - data['yx_spread'].rolling(window=window, min_periods=1).mean()
        )
    
    # Spread std and zscore with rolling windows INCLUDING current
    data['yx_spread_time_std_4hours'] = (
        data['yx_spread'].rolling(window=1410, min_periods=1).std().fillna(0) + std_reg_const
    )
    data['yx_spread_time_zscore_4hours'] = data['yx_spread_time_mean_4hours'] / data['yx_spread_time_std_4hours']
    
    # Spread lags (excluding current)
    data['yx_spread_time_mean_10min_lag_1hour'] = data['yx_spread_time_mean_10min'].shift(360)
    data['yx_spread_time_mean_2hours_lag_20min'] = data['yx_spread_time_mean_2hours'].shift(120)
    
    # Spread EWMA INCLUDING current
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'yx_spread_dayly_ewma_10min'] = (
            day_data['yx_spread'].ewm(halflife=60, adjust=False).mean()
        )
        data.loc[df_mask, 'yx_spread_dayly_ewma_1hour'] = (
            day_data['yx_spread'].ewm(halflife=360, adjust=False).mean()
        )
    
    data['yx_spread_dayly_ewma_10min'] = data['yx_spread'] - data['yx_spread_dayly_ewma_10min']
    data['yx_spread_dayly_ewma_1hour'] = data['yx_spread'] - data['yx_spread_dayly_ewma_1hour']
    data['yx_spread_ewma_prodpair_1hour_10min'] = (
        data.yx_spread_dayly_ewma_1hour * data.yx_spread_dayly_ewma_10min)
    
    # Clean up temporary columns
    temp_cols = [
        'xy_garmonic_dayly_ewma_1hour', 'xy_garmonic_dayly_ewma_2hours',
        'xy_square_time_mean_10min', 'xy_square_time_std_10min',
        'yx_spread_dayly_ewma_10min', 'yx_spread_dayly_ewma_1hour',
        'yprice_time_std_1hour', 'yprice_time_std_2hours',
        'yx_spread_time_std_4hours', 'xy_relation_time_std_1hour',
        'xy_relation_time_std_2hours'
    ]
    
    for col in temp_cols:
        if col in data.columns:
            data.drop(col, axis=1, inplace=True)
    
    print(f"Feature engineering completed (NO DATA LEAKAGE - {'TRAIN' if is_train else 'TEST'})!")
    return data

# %% Data Initialization
def init_data_single(fname):
    """Initialize and preprocess a single dataset"""
    print(f'Loading file: {fname}...')
    data = pd.read_csv(fname)
    
    data['xprice'] -= 127  # WARNING: Domain-specific adjustment
    data['yprice'] -= 146  # WARNING: Domain-specific adjustment
    
    # Create derived price features
    data['yx_spread'] = data.yprice - data.xprice
    data['yx_relation'] = data.yprice / data.xprice
    data['xy_relation'] = data.xprice / data.yprice
    data['xy_square'] = np.sqrt(data.xprice ** 2 + data.yprice ** 2) / 2
    data['xy_geom'] = np.sqrt(data.xprice * data.yprice)
    data['xy_garmonic'] = 2 / (1 / data.xprice + 1 / data.yprice)
    
    # Process timestamps
    data['timestamp'] = data['timestamp'] // 1000
    data['timestamp'] = data['timestamp'].apply(lambda stamp: datetime.fromtimestamp(stamp))
    data['timestamp'] = data['timestamp'] - pd.Timedelta(hours=1)
    data.index = data['timestamp']
    
    # Add time-based features
    data['weekday'] = data.timestamp.dt.weekday
    data['is_end_of_week'] = (data.timestamp.dt.weekday >= 2).astype(int)
    
    data['day'] = (data.timestamp.dt.date - data.timestamp.dt.date.min()).apply(lambda x: int(x.days))
    day_close_time = data.day.map(data.groupby('day').timestamp.max())
    data['periods_before_closing'] = (day_close_time - data.timestamp).apply(lambda x: x.seconds // 10)
    day_open_time = data.day.map(data.groupby('day').timestamp.min())
    data['periods_after_opening'] = (data.timestamp - day_open_time).apply(lambda x: x.seconds // 10)
    
    return data

def selected_features_extractor_separate(train_data_path, test_data_path):
    """
    Extract features separately for train and test to avoid leakage.
    This is the SAFE approach - features are computed independently.
    
    Parameters:
    train_data_path: Path to training data (can be None if only processing test)
    test_data_path: Path to test data (can be None if only processing train)
    """
    train = None
    test = None
    
    # Process training data if provided
    if train_data_path is not None:
        train_data = init_data_single(train_data_path)
        train_data = create_all_features(train_data, is_train=True)
        
        usecols = selected_cols + ['returns', 'periods_before_closing']
        train = train_data[usecols].iloc[droprows:].copy()
    
    # Process test data if provided
    if test_data_path is not None:
        test_data = init_data_single(test_data_path)
        test_data = create_all_features(test_data, is_train=False)
        
        # Test might not have 'returns' column
        test_cols = selected_cols + ['periods_before_closing']
        if 'returns' in test_data.columns:
            test_cols.append('returns')
        test = test_data[test_cols].copy()
    
    return train, test

# %% Model Training Functions
def normalize_train(df):
    """Normalize training data"""
    extended_cols = selected_cols + ['returns', 'periods_before_closing']
    norm_train = df[extended_cols].reset_index(drop=True).copy()
    norm_mean = norm_train[selected_cols].mean()
    norm_std = norm_train[selected_cols].std() + normalization_std_reg
    norm_train.loc[:,selected_cols] = (norm_train[selected_cols] - norm_mean) / norm_std
    return norm_train

def normalize_train_test(train, test):
    """Normalize train and test data using training statistics"""
    train_extended_cols = selected_cols + ['returns', 'periods_before_closing']
    test_extended_cols = selected_cols + ['periods_before_closing']
    
    norm_train = train[train_extended_cols].reset_index(drop=True).copy()
    norm_test = test[test_extended_cols].reset_index(drop=True).copy()
    
    norm_mean = norm_train[selected_cols].mean()
    norm_std = norm_train[selected_cols].std() + normalization_std_reg
    
    norm_train.loc[:,selected_cols] = (norm_train[selected_cols] - norm_mean) / norm_std
    norm_test.loc[:,selected_cols] = (norm_test[selected_cols] - norm_mean) / norm_std
    return norm_train, norm_test

# %% Model Estimation Function
def modelEstimate(train_data_path):
    """
    Train the XGBoost model on the training data.
    
    Parameters:
    train_data_path (str): Path to the training CSV file
    
    Returns:
    dict: Dictionary containing the trained model and normalization parameters
    """
    print('--------->Start Estimation with XGBoost....')
    
    # Extract features (train only, no test contamination)
    train, _ = selected_features_extractor_separate(train_data_path, None)
    
    # Remove any rows with NaN values
    train = train.dropna()
    
    # Normalize training data
    print('----->Normalization....')
    norm_train = normalize_train(train)
    
    # Calculate normalization parameters for later use
    norm_mean = train[selected_cols].mean()
    norm_std = train[selected_cols].std() + normalization_std_reg
    
    # Validate model
    print('----->Validation with XGBoost....')
    model = XGBRegressor(**xgb_params)
    print(validate_model_by_pentate(model, norm_train, selected_cols, 0))
    
    # Fit final model
    print('----->Fitting XGBoost model....')
    print(f'Using XGBoost parameters: {xgb_params}')
    
    # Create model with early stopping included in parameters
    xgb_params_with_early_stopping = xgb_params.copy()
    xgb_params_with_early_stopping['early_stopping_rounds'] = 50
    xgb_params_with_early_stopping['eval_metric'] = 'rmse'
    
    model = XGBRegressor(**xgb_params_with_early_stopping)
    
    # Optional: Use early stopping with validation set
    # Split off a validation set for early stopping
    val_size = int(len(norm_train) * 0.1)
    train_X = norm_train[selected_cols].iloc[:-val_size]
    train_y = norm_train.returns.iloc[:-val_size]
    val_X = norm_train[selected_cols].iloc[-val_size:]
    val_y = norm_train.returns.iloc[-val_size:]
    
    # Fit with validation set
    model.fit(
        train_X, train_y,
        eval_set=[(val_X, val_y)],
        verbose=False
    )
    
    # Check if model has best_iteration attribute (depends on XGBoost version)
    if hasattr(model, 'best_iteration'):
        print(f'Best iteration: {model.best_iteration}')
    if hasattr(model, 'best_score'):
        print(f'Best score: {model.best_score:.6f}')
    
    # Print feature importances
    print('----->Calculating feature importances (XGBoost)...')
    print_importances_xgb(model, selected_cols)
    
    # Return model and parameters
    return {
        'model': model,
        'norm_mean': norm_mean,
        'norm_std': norm_std,
        'xgb_params': xgb_params
    }

# %% Model Forecast Function
def modelForecast(test_data_path, model_params, train_data_path=None):
    """
    Make predictions on test data using the trained XGBoost model.
    
    Parameters:
    test_data_path (str): Path to the test CSV file
    model_params (dict): Dictionary containing the trained model and normalization parameters
    train_data_path (str): Not used in this version (features computed separately)
    
    Returns:
    dict: Dictionary containing predictions and evaluation metrics (if test data has returns)
    """
    print('--------->Starting Forecasting with XGBoost....')
    
    # Extract features for test only (no train contamination)
    _, test = selected_features_extractor_separate(None, test_data_path)
    
    # Remove any rows with NaN values
    test = test.dropna()
    
    # Normalize test data using training statistics
    print('----->Normalization....')
    test_extended_cols = selected_cols + ['periods_before_closing']
    
    # Check if test data has returns column for evaluation
    has_returns = 'returns' in test.columns
    if has_returns:
        test_extended_cols.append('returns')
    
    norm_test = test[test_extended_cols].reset_index(drop=True).copy()
    norm_test.loc[:,selected_cols] = (norm_test[selected_cols] - model_params['norm_mean']) / model_params['norm_std']
    
    # Make predictions
    print('----->Prediction with XGBoost....')
    predicted = model_params['model'].predict(norm_test[selected_cols])
    
    # Set predictions to 0 at closing periods
    predicted[norm_test.periods_before_closing == 0] = 0
    
    # Calculate evaluation metrics if test data has returns
    results = {'predictions': predicted}
    
    if has_returns:
        print('----->Evaluation....')
        # Calculate MSE
        mse = mean_squared_error(norm_test.returns, predicted)
        # Calculate R-squared
        r2 = rsquared(norm_test.returns, predicted)
        
        results['mse'] = mse
        results['r2'] = r2
        
        print(f'Test MSE: {mse:.6f}')
        print(f'Test R-squared: {r2:.6f}')
        print(f'Test R-squared (x100): {r2*100:.2f}')
    else:
        print('No returns column in test data - evaluation metrics not calculated')
    
    return results

# %% Example Usage
if __name__ == "__main__":
    # Train the model
    train_file_path = '/Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/train.csv'  # Replace with your actual file path
    
    print("="*70)
    print("Training XGBoost Model for Financial Time Series Prediction")
    print("="*70)
    
    model_params = modelEstimate(train_file_path)
    
    # Save model for later use
    with open('xgboost_model_params.pkl', 'wb') as f:
        pickle.dump(model_params, f)
    print("\nModel saved to xgboost_model_params.pkl")
    
    # Make predictions on test data
    test_file_path = '/Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/test.csv'  # Replace with your actual file path
    
    print("\n" + "="*70)
    print("Making Predictions on Test Data")
    print("="*70)
    
    # Load model if needed
    # with open('xgboost_model_params.pkl', 'rb') as f:
    #     model_params = pickle.load(f)
    
    results = modelForecast(test_file_path, model_params)
    
    # Extract predictions
    predictions = results['predictions']
    
    # Display results
    print(f"\nFirst 10 predictions: {predictions[:10]}")
    print(f"Total predictions: {len(predictions)}")
    
    # Display evaluation metrics if available
    if 'mse' in results:
        print(f"\nEvaluation Metrics:")
        print(f"MSE: {results['mse']:.6f}")
        print(f"R-squared: {results['r2']:.6f}")
        print(f"R-squared (x100): {results['r2']*100:.2f}")
    
    # Save predictions to CSV
    predictions_df = pd.DataFrame({'predictions': predictions})
    predictions_df.to_csv('xgboost_predictions.csv', index=False)
    print("\nPredictions saved to xgboost_predictions.csv")
    
    # Additional analysis: Get feature importance plot data
    print("\n" + "="*70)
    print("Top 10 Most Important Features:")
    print("="*70)
    
    feature_importance = model_params['model'].feature_importances_
    importance_dict = dict(zip(selected_cols, feature_importance))
    sorted_features = sorted(importance_dict.items(), key=lambda x: -x[1])[:10]
    
    for i, (name, importance) in enumerate(sorted_features, 1):
        print(f"{i:2}. {name:40} {importance:.6f}")
    
    print("\n" + "="*70)
    print("XGBoost Model Training and Prediction Complete!")
    print("="*70)

Training XGBoost Model for Financial Time Series Prediction
--------->Start Estimation with XGBoost....
Loading file: /Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/train.csv...
Creating all features (NO LEAKAGE - TRAIN)...
Creating opening/closing difference features...
Creating log features...
Creating expanding mean difference...
Creating time-based rolling features (TRULY FIXED)...
Creating intraday EWMA features...
Creating lag features...
Creating RSI features...
Creating EWMA on time means...
Creating XY combined features...
Creating YX spread features...
Feature engineering completed (NO DATA LEAKAGE - TRAIN)!
----->Normalization....
----->Validation with XGBoost....
     train_50_percent  train_60_percent  train_70_percent  train_80_percent  \
mse          0.037720          0.023819          0.019280          0.035759   
r2           1.266047          1.117368          0.000252          0.174544   

     train_90_percent  min_stats  max_stats      

# Elastic Net CV with all features (better r squared than ridge)

In [85]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from copy import copy
import scipy
import pickle

# %% Configuration
# Configuration parameters
droprows = 7050
std_reg_const = 0.1
normalization_std_reg = 0.0001

# ElasticNetCV parameters
elasticnet_params = {
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99],  # Mix of L1 and L2 penalties
    'alphas': np.logspace(-4, 2, 100),  # Range of regularization strengths to try
    'cv': 5,  # 5-fold cross-validation
    'max_iter': 10000,  # Maximum iterations
    'tol': 0.0001,  # Tolerance for optimization
    'random_state': 42,
    'selection': 'cyclic',  # Feature selection method
    'fit_intercept': True
}

selected_cols = [
    
    # Time-based features
    'is_end_of_week',
    'weekday',
    
    # Opening/Closing difference features
    'xdiff_from_closing',
    'xdiff_from_opening',
    'ydiff_from_closing',
    
    # X-price features
    'xlog',
    'xlog_dayly_ewma_10min',
    'xprice_dayly_ewma_4hour',
    'xprice_lag_4hours',
    'xprice_time_mean_4hours',
    'xprice_time_mean_1hour_lag_4hours',
    'xprice_time_mean_1min',
    'xprice_time_mean_10min_dayly_ewma_10min',
    'xprice_time_mean_1min_rsi_1min',
    
    # Y-price features
    'ylog_dayly_ewma_1hour',
    'yprice_dayly_ewma_10min',
    'yprice_lag_4hours',
    'yprice_ewma_difpair_10min_4hour',
    'yprice_expanding_mean_diff', # FIXED: renamed from yprice_full_history_diff
    'yprice_time_mean_1hour',
    'yprice_time_mean_1hour_lag_4hours',
    'yprice_time_mean_1hour_lag_1workweek',
    'yprice_time_mean_10min',
    'yprice_time_mean_10min_dayly_ewma_4hour',
    'yprice_time_mean_10min_lag_10min',
    'yprice_time_mean_10min_rsi_1hour',
    'yprice_time_mean_2hours',
    'yprice_time_zscore_1hour',
    'yprice_time_zscore_2hours',
    
    # XY combined features
    'xy_garmonic_ewma_prodpair_2hours_1hour',
    'xy_garmonic_time_std_4hours',
    'xy_geom_time_mean_1hour_lag_20min',
    'xy_geom_time_mean_1min',
    'xy_geom_time_mean_10min_dayly_ewma_1min',
    'xy_geom_time_mean_2hours_dayly_ewma_20min',
    'xy_square_time_zscore_10min',
    
    # YX spread features
    'yx_spread_ewma_prodpair_1hour_10min',
    'yx_spread_time_mean_10min_lag_1hour',
    'yx_spread_time_mean_2hours_lag_20min',
    'yx_spread_time_zscore_4hours',
]


# %% Helper Functions
def print_importances_elasticnet(model, selected_cols):
    """Print feature importances for ElasticNet model sorted by absolute coefficient"""
    weights_sum = sum(map(abs, model.coef_))
    
    print("\nFeature Importances (ElasticNet Coefficients):")
    print("-" * 70)
    
    # Create a list of (name, weight) tuples and sort by absolute weight
    feature_weights = list(zip(selected_cols, model.coef_))
    sorted_features = sorted(feature_weights, key=lambda x: -abs(x[1]))
    
    # Print non-zero features
    non_zero_count = sum(1 for _, weight in feature_weights if abs(weight) > 1e-10)
    print(f"Non-zero features: {non_zero_count}/{len(selected_cols)}")
    print(f"Selected alpha: {model.alpha_:.6f}")
    print(f"Selected l1_ratio: {model.l1_ratio_:.3f}")
    print("-" * 70)
    
    for name, weight in sorted_features:
        if abs(weight) > 1e-10:  # Only show non-zero weights
            percent_weight = abs(weight) / weights_sum if weights_sum > 0 else 0
            print('{:40} {:.2%} {:15.6f}'.format(name, percent_weight, weight))

def rsquared(x, y):
    """Return R^2 where x and y are array-like."""
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

def rsiFunc(prices, n=14):
    """Calculate RSI (Relative Strength Index)"""
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1]
        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n
        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)
    return rsi

# %% Time Series Validation Functions
def time_split(data, valid_ratio, test_ratio):
    """Split time series data into train, validation, and test sets"""
    n_valid = max(1, int(data.shape[0] * valid_ratio))
    n_test = max(1, int(data.shape[0] * test_ratio))
    n_train = data.shape[0] - n_valid - n_test
    
    train = data.iloc[:n_train].reset_index(drop=True).copy()
    valid = data.iloc[n_train:-n_test].reset_index(drop=True).copy()
    test = data.iloc[-n_test:].reset_index(drop=True).copy()
    merged_test = pd.concat([valid, test], ignore_index=True)
    return train, valid, test

def validate_model_by_pentate(model_class, model_params, source_data, base_cols, droprows=0):
    """Validate model using 5-fold time series cross-validation"""
    df = source_data.copy()
    selected_cols = base_cols.copy()
    helper_cols = list(set(selected_cols + ['periods_before_closing', 'returns']))
    metrics_dict = {}
    
    for step in range(5, 10):
        n_train = int(df.shape[0] * step // 10)
        n_test = int(df.shape[0] * (step + 1) // 10)
        train = df.iloc[:n_train].reset_index(drop=True).copy()
        test = df.iloc[n_train:n_test].reset_index(drop=True).copy()
        train.drop(np.arange(droprows), inplace=True)
        train.dropna(inplace=True)

        # Create and fit a new model for each fold
        fold_model = model_class(**model_params)
        fold_model.fit(train[selected_cols], train.returns)
        predicted = fold_model.predict(test[selected_cols])
        predicted[test.periods_before_closing == 0] = 0

        current_mse = mean_squared_error(test.returns, predicted)
        current_r2 = rsquared(test.returns, predicted) * 100
        metrics_dict['train_{}_percent'.format(step * 10)] = {
            'mse': current_mse,
            'r2': current_r2
        }
    
    report = pd.DataFrame(metrics_dict)
    report['min_stats'] = report.iloc[:,:5].min(1).astype(np.float32)
    report['max_stats'] = report.iloc[:,:5].max(1).astype(np.float32)
    report['avg'] = report.mean(1).astype(np.float32)
    return report

# %% TRULY FIXED Feature Engineering Function
def create_all_features(data, is_train=True):
    """
    TRULY FIXED: Feature engineering with NO data leakage.
    All features use strictly backward-looking rolling windows.
    Current observation at time t IS included (since we predict t+60).
    No resample+ffill pattern that causes intra-bin leakage.
    """
    print(f"Creating all features (NO LEAKAGE - {'TRAIN' if is_train else 'TEST'})...")
    
    # Pre-calculate commonly used values
    days = data.day.unique()
    
    # Opening/Closing differences (using previous day's close)
    print("Creating opening/closing difference features...")
    close_price_per_day_y = data.groupby('day').timestamp.max().shift(1).map(
        data[['timestamp', 'yprice']].set_index('timestamp').yprice)
    data['ydiff_from_closing'] = (data.yprice - data.day.map(close_price_per_day_y)).fillna(0)
    
    close_price_per_day_x = data.groupby('day').timestamp.max().shift(1).map(
        data[['timestamp', 'xprice']].set_index('timestamp').xprice)
    data['xdiff_from_closing'] = (data.xprice - data.day.map(close_price_per_day_x)).fillna(0)
    
    open_price_per_day_x = data.groupby('day').timestamp.min().map(
        data[['timestamp', 'xprice']].set_index('timestamp').xprice)
    data['xdiff_from_opening'] = data.xprice - data.day.map(open_price_per_day_x)
    
    # Log features
    print("Creating log features...")
    data['xlog'] = data.xprice.apply(np.log1p)
    data['ylog'] = data.yprice.apply(np.log1p)
    
    # Expanding mean that only uses past data (including current)
    print("Creating expanding mean difference...")
    data['yprice_expanding_mean_diff'] = data['yprice'] - data['yprice'].expanding(min_periods=1).mean()
    
    # FIXED: Time-based rolling features with backward-looking windows INCLUDING current
    print("Creating time-based rolling features (TRULY FIXED)...")
    time_windows = {
        6: '1min', 60: '10min', 360: '1hour', 720: '2hours', 1410: '4hours', 2820: '1workweek'
    }
    
    # Create rolling means with backward-looking windows INCLUDING current observation
    for window, name in time_windows.items():
        if window in [6, 60, 360, 1410]:  # xprice windows
            colname = f'xprice_time_mean_{name}'
            # Rolling includes current observation (no shift)
            data[colname] = data['xprice'] - data['xprice'].rolling(
                window=window, min_periods=1).mean()
        
        if window in [60, 360, 720]:  # yprice windows
            colname = f'yprice_time_mean_{name}'
            # Rolling includes current observation (no shift)
            data[colname] = data['yprice'] - data['yprice'].rolling(
                window=window, min_periods=1).mean()
    
    # Create rolling std for yprice with backward-looking windows INCLUDING current
    for window in [360, 720]:
        name = time_windows[window]
        colname = f'yprice_time_std_{name}'
        # Rolling includes current observation (no shift)
        data[colname] = data['yprice'].rolling(
            window=window, min_periods=1).std().fillna(0) + std_reg_const
    
    # Z-scores for yprice
    data['yprice_time_zscore_1hour'] = data.yprice_time_mean_1hour / data.yprice_time_std_1hour
    data['yprice_time_zscore_2hours'] = data.yprice_time_mean_2hours / data.yprice_time_std_2hours
    
    # Intraday EWMA features (backward-looking INCLUDING current)
    print("Creating intraday EWMA features...")
    ewma_configs = [
        ('xprice', [24], ['4hour']),
        ('xlog', [60], ['10min']),
        ('ylog', [360], ['1hour']),
        ('yprice', [24, 60], ['4hour', '10min'])
    ]
    
    for col, windows, names in ewma_configs:
        for day in days:
            df_mask = (data.day == day)
            day_data = data.loc[df_mask].copy()
            
            for window, name in zip(windows, names):
                colname = f'{col}_dayly_ewma_{name}'
                # EWMA includes current observation (no shift)
                ewm = day_data[col].ewm(halflife=window, adjust=False).mean()
                
                if col in ['xprice', 'yprice']:
                    data.loc[df_mask, colname] = day_data[col] - ewm
                elif col in ['xlog', 'ylog']:
                    data.loc[df_mask, colname] = day_data[col.replace('log', 'price')] - ewm
    
    # EWMA difference pair
    data['yprice_ewma_difpair_10min_4hour'] = data.yprice_dayly_ewma_10min - data.yprice_dayly_ewma_4hour
    
    # Lag features (inherently backward-looking, excludes current)
    print("Creating lag features...")
    lag_configs = [
        ('xprice', 1410, '4hours'),
        ('yprice', 1410, '4hours'),
        ('xprice_time_mean_1hour', 1410, '4hours'),
        ('yprice_time_mean_1hour', 1410, '4hours'),
        ('yprice_time_mean_1hour', 2820, '1workweek'),
        ('yprice_time_mean_10min', 60, '10min')
    ]
    
    for col, lag, name in lag_configs:
        data[f'{col}_lag_{name}'] = data[col].shift(lag)
    
    # RSI features (calculated on backward-looking data including current)
    print("Creating RSI features...")
    # RSI needs some history, so we calculate it on the rolling means
    if 'xprice_time_mean_1min' in data.columns:
        data['xprice_time_mean_1min_rsi_1min'] = rsiFunc(data['xprice_time_mean_1min'].fillna(0).values, 6)
    data['yprice_time_mean_10min_rsi_1hour'] = rsiFunc(data['yprice_time_mean_10min'].fillna(0).values, 360)
    
    # Additional EWMA on time means
    print("Creating EWMA on time means...")
    ewma_on_means = [
        ('xprice_time_mean_10min', 60, '10min'),
        ('yprice_time_mean_10min', 24, '4hour')
    ]
    
    for col, window, name in ewma_on_means:
        for day in days:
            df_mask = (data.day == day)
            day_data = data.loc[df_mask].copy()
            colname = f'{col}_dayly_ewma_{name}'
            # EWMA includes current observation (no shift)
            ewm = day_data[col].ewm(halflife=window, adjust=False).mean()
            data.loc[df_mask, colname] = ewm
    
    # XY Combined features with backward-looking windows INCLUDING current
    print("Creating XY combined features...")
    
    # XY Harmonic std - use rolling window INCLUDING current
    data['xy_garmonic_time_std_4hours'] = (
        data['xy_garmonic'].rolling(window=1410, min_periods=1).std().fillna(0) + std_reg_const
    )
    
    # Harmonic EWMA
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'xy_garmonic_dayly_ewma_1hour'] = (
            day_data['xy_garmonic'].ewm(halflife=360, adjust=False).mean()
        )
        data.loc[df_mask, 'xy_garmonic_dayly_ewma_2hours'] = (
            day_data['xy_garmonic'].ewm(halflife=720, adjust=False).mean()
        )
    
    data['xy_garmonic_dayly_ewma_1hour'] = data['xy_garmonic'] - data['xy_garmonic_dayly_ewma_1hour']
    data['xy_garmonic_dayly_ewma_2hours'] = data['xy_garmonic'] - data['xy_garmonic_dayly_ewma_2hours']
    data['xy_garmonic_ewma_prodpair_2hours_1hour'] = (
        data.xy_garmonic_dayly_ewma_2hours * data.xy_garmonic_dayly_ewma_1hour)
    
    # XY Geometric features with rolling windows INCLUDING current
    for window, name in [(6, '1min'), (60, '10min'), (360, '1hour'), (720, '2hours')]:
        colname = f'xy_geom_time_mean_{name}'
        # Rolling includes current observation (no shift)
        data[colname] = data['xy_geom'] - data['xy_geom'].rolling(
            window=window, min_periods=1).mean()
    
    # Geometric lags and EWMA
    data['xy_geom_time_mean_1hour_lag_20min'] = data['xy_geom_time_mean_1hour'].shift(120)
    
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'xy_geom_time_mean_10min_dayly_ewma_1min'] = (
            day_data['xy_geom_time_mean_10min'].ewm(halflife=6, adjust=False).mean()
        )
        data.loc[df_mask, 'xy_geom_time_mean_2hours_dayly_ewma_20min'] = (
            day_data['xy_geom_time_mean_2hours'].ewm(halflife=120, adjust=False).mean()
        )
    
    # XY Relation std with rolling windows INCLUDING current
    for window, name in [(360, '1hour'), (720, '2hours')]:
        colname = f'xy_relation_time_std_{name}'
        data[colname] = (
            data['xy_relation'].rolling(window=window, min_periods=1).std().fillna(0) + std_reg_const
        )
    
    # XY Square zscore with rolling windows INCLUDING current
    data['xy_square_time_mean_10min'] = (
        data['xy_square'] - data['xy_square'].rolling(window=60, min_periods=1).mean()
    )
    data['xy_square_time_std_10min'] = (
        data['xy_square'].rolling(window=60, min_periods=1).std().fillna(0) + std_reg_const
    )
    data['xy_square_time_zscore_10min'] = data['xy_square_time_mean_10min'] / data['xy_square_time_std_10min']
    
    # YX Spread features with rolling windows INCLUDING current
    print("Creating YX spread features...")
    for window, name in [(60, '10min'), (720, '2hours'), (1410, '4hours')]:
        colname = f'yx_spread_time_mean_{name}'
        data[colname] = (
            data['yx_spread'] - data['yx_spread'].rolling(window=window, min_periods=1).mean()
        )
    
    # Spread std and zscore with rolling windows INCLUDING current
    data['yx_spread_time_std_4hours'] = (
        data['yx_spread'].rolling(window=1410, min_periods=1).std().fillna(0) + std_reg_const
    )
    data['yx_spread_time_zscore_4hours'] = data['yx_spread_time_mean_4hours'] / data['yx_spread_time_std_4hours']
    
    # Spread lags (excluding current)
    data['yx_spread_time_mean_10min_lag_1hour'] = data['yx_spread_time_mean_10min'].shift(360)
    data['yx_spread_time_mean_2hours_lag_20min'] = data['yx_spread_time_mean_2hours'].shift(120)
    
    # Spread EWMA INCLUDING current
    for day in days:
        df_mask = (data.day == day)
        day_data = data.loc[df_mask].copy()
        
        # EWMA includes current observation (no shift)
        data.loc[df_mask, 'yx_spread_dayly_ewma_10min'] = (
            day_data['yx_spread'].ewm(halflife=60, adjust=False).mean()
        )
        data.loc[df_mask, 'yx_spread_dayly_ewma_1hour'] = (
            day_data['yx_spread'].ewm(halflife=360, adjust=False).mean()
        )
    
    data['yx_spread_dayly_ewma_10min'] = data['yx_spread'] - data['yx_spread_dayly_ewma_10min']
    data['yx_spread_dayly_ewma_1hour'] = data['yx_spread'] - data['yx_spread_dayly_ewma_1hour']
    data['yx_spread_ewma_prodpair_1hour_10min'] = (
        data.yx_spread_dayly_ewma_1hour * data.yx_spread_dayly_ewma_10min)
    
    # Clean up temporary columns
    temp_cols = [
        'xy_garmonic_dayly_ewma_1hour', 'xy_garmonic_dayly_ewma_2hours',
        'xy_square_time_mean_10min', 'xy_square_time_std_10min',
        'yx_spread_dayly_ewma_10min', 'yx_spread_dayly_ewma_1hour',
        'yprice_time_std_1hour', 'yprice_time_std_2hours',
        'yx_spread_time_std_4hours', 'xy_relation_time_std_1hour',
        'xy_relation_time_std_2hours'
    ]
    
    for col in temp_cols:
        if col in data.columns:
            data.drop(col, axis=1, inplace=True)
    
    print(f"Feature engineering completed (NO DATA LEAKAGE - {'TRAIN' if is_train else 'TEST'})!")
    return data

# %% Data Initialization
def init_data_single(fname):
    """Initialize and preprocess a single dataset"""
    print(f'Loading file: {fname}...')
    data = pd.read_csv(fname)
    
    data['xprice'] -= 127  # WARNING: Domain-specific adjustment
    data['yprice'] -= 146  # WARNING: Domain-specific adjustment
    
    # Create derived price features
    data['yx_spread'] = data.yprice - data.xprice
    data['yx_relation'] = data.yprice / data.xprice
    data['xy_relation'] = data.xprice / data.yprice
    data['xy_square'] = np.sqrt(data.xprice ** 2 + data.yprice ** 2) / 2
    data['xy_geom'] = np.sqrt(data.xprice * data.yprice)
    data['xy_garmonic'] = 2 / (1 / data.xprice + 1 / data.yprice)
    
    # Process timestamps
    data['timestamp'] = data['timestamp'] // 1000
    data['timestamp'] = data['timestamp'].apply(lambda stamp: datetime.fromtimestamp(stamp))
    data['timestamp'] = data['timestamp'] - pd.Timedelta(hours=1)
    data.index = data['timestamp']
    
    # Add time-based features
    data['weekday'] = data.timestamp.dt.weekday
    data['is_end_of_week'] = (data.timestamp.dt.weekday >= 2).astype(int)
    
    data['day'] = (data.timestamp.dt.date - data.timestamp.dt.date.min()).apply(lambda x: int(x.days))
    day_close_time = data.day.map(data.groupby('day').timestamp.max())
    data['periods_before_closing'] = (day_close_time - data.timestamp).apply(lambda x: x.seconds // 10)
    day_open_time = data.day.map(data.groupby('day').timestamp.min())
    data['periods_after_opening'] = (data.timestamp - day_open_time).apply(lambda x: x.seconds // 10)
    
    return data

def selected_features_extractor_separate(train_data_path, test_data_path):
    """
    Extract features separately for train and test to avoid leakage.
    This is the SAFE approach - features are computed independently.
    
    Parameters:
    train_data_path: Path to training data (can be None if only processing test)
    test_data_path: Path to test data (can be None if only processing train)
    """
    train = None
    test = None
    
    # Process training data if provided
    if train_data_path is not None:
        train_data = init_data_single(train_data_path)
        train_data = create_all_features(train_data, is_train=True)
        
        usecols = selected_cols + ['returns', 'periods_before_closing']
        train = train_data[usecols].iloc[droprows:].copy()
    
    # Process test data if provided
    if test_data_path is not None:
        test_data = init_data_single(test_data_path)
        test_data = create_all_features(test_data, is_train=False)
        
        # Test might not have 'returns' column
        test_cols = selected_cols + ['periods_before_closing']
        if 'returns' in test_data.columns:
            test_cols.append('returns')
        test = test_data[test_cols].copy()
    
    return train, test

# %% Model Training Functions
def normalize_train(df):
    """Normalize training data"""
    extended_cols = selected_cols + ['returns', 'periods_before_closing']
    norm_train = df[extended_cols].reset_index(drop=True).copy()
    norm_mean = norm_train[selected_cols].mean()
    norm_std = norm_train[selected_cols].std() + normalization_std_reg
    norm_train.loc[:,selected_cols] = (norm_train[selected_cols] - norm_mean) / norm_std
    return norm_train

def normalize_train_test(train, test):
    """Normalize train and test data using training statistics"""
    train_extended_cols = selected_cols + ['returns', 'periods_before_closing']
    test_extended_cols = selected_cols + ['periods_before_closing']
    
    norm_train = train[train_extended_cols].reset_index(drop=True).copy()
    norm_test = test[test_extended_cols].reset_index(drop=True).copy()
    
    norm_mean = norm_train[selected_cols].mean()
    norm_std = norm_train[selected_cols].std() + normalization_std_reg
    
    norm_train.loc[:,selected_cols] = (norm_train[selected_cols] - norm_mean) / norm_std
    norm_test.loc[:,selected_cols] = (norm_test[selected_cols] - norm_mean) / norm_std
    return norm_train, norm_test

# %% Model Estimation Function
def modelEstimate(train_data_path):
    """
    Train the ElasticNetCV model on the training data.
    
    Parameters:
    train_data_path (str): Path to the training CSV file
    
    Returns:
    dict: Dictionary containing the trained model and normalization parameters
    """
    print('--------->Start Estimation with ElasticNetCV....')
    
    # Extract features (train only, no test contamination)
    train, _ = selected_features_extractor_separate(train_data_path, None)
    
    # Remove any rows with NaN values
    train = train.dropna()
    
    # Normalize training data
    print('----->Normalization....')
    norm_train = normalize_train(train)
    
    # Calculate normalization parameters for later use
    norm_mean = train[selected_cols].mean()
    norm_std = train[selected_cols].std() + normalization_std_reg
    
    # Validate model (using simpler parameters for validation)
    print('----->Validation with ElasticNetCV....')
    simple_params = {
        'l1_ratio': 0.5,  # Use single value for validation
        'cv': 5,
        'max_iter': 10000,
        'random_state': 42
    }
    print(validate_model_by_pentate(ElasticNetCV, simple_params, norm_train, selected_cols, 0))
    
    # Fit final model with full parameter search
    print('----->Fitting ElasticNetCV model with cross-validation....')
    print(f'Testing l1_ratios: {elasticnet_params["l1_ratio"]}')
    print(f'Testing {len(elasticnet_params["alphas"])} alpha values from {elasticnet_params["alphas"].min():.6f} to {elasticnet_params["alphas"].max():.6f}')
    print(f'Using {elasticnet_params["cv"]}-fold cross-validation')
    
    model = ElasticNetCV(
        l1_ratio=elasticnet_params['l1_ratio'],
        alphas=elasticnet_params['alphas'],
        cv=elasticnet_params['cv'],
        max_iter=elasticnet_params['max_iter'],
        tol=elasticnet_params['tol'],
        random_state=elasticnet_params['random_state'],
        selection=elasticnet_params['selection'],
        fit_intercept=elasticnet_params['fit_intercept']
    )
    
    # Fit the model
    model.fit(norm_train[selected_cols], norm_train.returns)
    
    # Print selected hyperparameters
    print(f'\n----->Optimal hyperparameters found:')
    print(f'Best alpha (regularization strength): {model.alpha_:.6f}')
    print(f'Best l1_ratio (L1 vs L2 mix): {model.l1_ratio_:.3f}')
    print(f'  - L1 penalty weight: {model.l1_ratio_:.1%}')
    print(f'  - L2 penalty weight: {(1-model.l1_ratio_):.1%}')
    
    # Print feature importances
    print('\n----->Calculating feature importances (ElasticNet)...')
    print_importances_elasticnet(model, selected_cols)
    
    # Return model and parameters
    return {
        'model': model,
        'norm_mean': norm_mean,
        'norm_std': norm_std,
        'elasticnet_params': elasticnet_params,
        'selected_alpha': model.alpha_,
        'selected_l1_ratio': model.l1_ratio_
    }

# %% Model Forecast Function
def modelForecast(test_data_path, model_params, train_data_path=None):
    """
    Make predictions on test data using the trained ElasticNetCV model.
    
    Parameters:
    test_data_path (str): Path to the test CSV file
    model_params (dict): Dictionary containing the trained model and normalization parameters
    train_data_path (str): Not used in this version (features computed separately)
    
    Returns:
    dict: Dictionary containing predictions and evaluation metrics (if test data has returns)
    """
    print('--------->Starting Forecasting with ElasticNetCV....')
    
    # Extract features for test only (no train contamination)
    _, test = selected_features_extractor_separate(None, test_data_path)
    
    # Remove any rows with NaN values
    test = test.dropna()
    
    # Normalize test data using training statistics
    print('----->Normalization....')
    test_extended_cols = selected_cols + ['periods_before_closing']
    
    # Check if test data has returns column for evaluation
    has_returns = 'returns' in test.columns
    if has_returns:
        test_extended_cols.append('returns')
    
    norm_test = test[test_extended_cols].reset_index(drop=True).copy()
    norm_test.loc[:,selected_cols] = (norm_test[selected_cols] - model_params['norm_mean']) / model_params['norm_std']
    
    # Make predictions
    print('----->Prediction with ElasticNetCV....')
    predicted = model_params['model'].predict(norm_test[selected_cols])
    
    # Set predictions to 0 at closing periods
    predicted[norm_test.periods_before_closing == 0] = 0
    
    # Calculate evaluation metrics if test data has returns
    results = {'predictions': predicted}
    
    if has_returns:
        print('----->Evaluation....')
        # Calculate MSE
        mse = mean_squared_error(norm_test.returns, predicted)
        # Calculate R-squared
        r2 = rsquared(norm_test.returns, predicted)
        
        results['mse'] = mse
        results['r2'] = r2
        
        print(f'Test MSE: {mse:.6f}')
        print(f'Test R-squared: {r2:.6f}')
        print(f'Test R-squared (x100): {r2*100:.2f}')
    else:
        print('No returns column in test data - evaluation metrics not calculated')
    
    return results

# %% Example Usage
if __name__ == "__main__":
    # Train the model
    train_file_path = '/Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/train.csv'  # Replace with your actual file path
    
    print("="*70)
    print("Training ElasticNetCV Model for Financial Time Series Prediction")
    print("="*70)
    
    model_params = modelEstimate(train_file_path)
    
    # Save model for later use
    with open('elasticnet_model_params.pkl', 'wb') as f:
        pickle.dump(model_params, f)
    print("\nModel saved to elasticnet_model_params.pkl")
    
    # Make predictions on test data
    test_file_path = '/Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/test.csv'  # Replace with your actual file path
    
    print("\n" + "="*70)
    print("Making Predictions on Test Data")
    print("="*70)
    
    # Load model if needed
    # with open('elasticnet_model_params.pkl', 'rb') as f:
    #     model_params = pickle.load(f)
    
    results = modelForecast(test_file_path, model_params)
    
    # Extract predictions
    predictions = results['predictions']
    
    # Display results
    print(f"\nFirst 10 predictions: {predictions[:10]}")
    print(f"Total predictions: {len(predictions)}")
    
    # Display evaluation metrics if available
    if 'mse' in results:
        print(f"\nEvaluation Metrics:")
        print(f"MSE: {results['mse']:.6f}")
        print(f"R-squared: {results['r2']:.6f}")
        print(f"R-squared (x100): {results['r2']*100:.2f}")
    
    # Save predictions to CSV
    predictions_df = pd.DataFrame({'predictions': predictions})
    predictions_df.to_csv('elasticnet_predictions.csv', index=False)
    print("\nPredictions saved to elasticnet_predictions.csv")
    
    # Additional analysis: Show sparsity and feature selection
    print("\n" + "="*70)
    print("Model Sparsity Analysis:")
    print("="*70)
    
    coefficients = model_params['model'].coef_
    non_zero_coefs = np.sum(np.abs(coefficients) > 1e-10)
    sparsity = 1 - (non_zero_coefs / len(coefficients))
    
    print(f"Total features: {len(coefficients)}")
    print(f"Non-zero coefficients: {non_zero_coefs}")
    print(f"Zero coefficients: {len(coefficients) - non_zero_coefs}")
    print(f"Sparsity: {sparsity:.1%}")
    print(f"\nSelected alpha: {model_params['selected_alpha']:.6f}")
    print(f"Selected l1_ratio: {model_params['selected_l1_ratio']:.3f}")
    
    # Show top 10 most important features
    print("\n" + "="*70)
    print("Top 10 Most Important Features (by absolute coefficient):")
    print("="*70)
    
    feature_importance = list(zip(selected_cols, coefficients))
    sorted_features = sorted(feature_importance, key=lambda x: -abs(x[1]))[:10]
    
    for i, (name, coef) in enumerate(sorted_features, 1):
        print(f"{i:2}. {name:40} {coef:15.6f}")
    
    print("\n" + "="*70)
    print("ElasticNetCV Model Training and Prediction Complete!")
    print("="*70)

Training ElasticNetCV Model for Financial Time Series Prediction
--------->Start Estimation with ElasticNetCV....
Loading file: /Users/mazin/Desktop/super prep materials/GSAPred/Two-financial-instruments/train.csv...
Creating all features (NO LEAKAGE - TRAIN)...
Creating opening/closing difference features...
Creating log features...
Creating expanding mean difference...
Creating time-based rolling features (TRULY FIXED)...
Creating intraday EWMA features...
Creating lag features...
Creating RSI features...
Creating EWMA on time means...
Creating XY combined features...
Creating YX spread features...
Feature engineering completed (NO DATA LEAKAGE - TRAIN)!
----->Normalization....
----->Validation with ElasticNetCV....
     train_50_percent  train_60_percent  train_70_percent  train_80_percent  \
mse          0.030347          0.021255          0.017238          0.017891   
r2           1.167415          0.839517          0.001305          1.112515   

     train_90_percent  min_stats  